In [1]:
!pip install shap

In [2]:
# Basic utilities
import warnings
import itertools
import joblib

# Math & Data handling
import numpy as np
import pandas as pd
from scipy.stats import mannwhitneyu, randint, uniform, loguniform
from scipy.linalg import LinAlgError

# Visualisation
import seaborn as sns
import shap
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from itertools import combinations

# Scikit-learn core
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from sklearn.exceptions import ConvergenceWarning

# Models
from sklearn.linear_model import (
    ElasticNet, ElasticNetCV, LinearRegression, LassoLars, LassoCV
)
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

# Metrics
from sklearn.metrics import mean_squared_error, r2_score

# Statsmodels
import statsmodels.formula.api as smf

# Clustering
from sklearn.cluster import OPTICS

In [3]:
# Load data frames
df_bas = pd.read_excel("BAS_clean_v5.xlsx", engine="openpyxl")
df_predictor = pd.read_csv("environmental_features.csv")

In [4]:
# Create a new data frame with selected columns
selected_cols = [
    'Location', 'Season', 'Crop', 'Residue', 'Treatment',
    'N_in', 'N_out', 'P_in', 'P_out', 'K_in', 'K_out', 'S_in', 'S_out'
]

df_target = df_bas[selected_cols].copy()

In [5]:
# Get columns excluding keys
target_cols = set(df_target.columns) - {'Location', 'Season'}
predictor_cols = set(df_predictor.columns) - {'location', 'season'}

# Check for overlapping columns
overlap = target_cols & predictor_cols
print(f"Overlapping columns: {overlap if overlap else 'None'}")

Overlapping columns: None


In [6]:
# Perform left join
df_full = pd.merge(
    df_target,
    df_predictor,
    left_on=['Location', 'Season'],
    right_on=['location', 'season'],
    how='left'
)

# Drop redundant columns from the right table
df_full = df_full.drop(columns=['location', 'season'])

In [7]:
# View the merged data frame
pd.set_option('display.max_columns', None)
df_full.head()

,Location,Season,Crop,Residue,Treatment,N_in,N_out,P_in,P_out,K_in,K_out,S_in,S_out,PAR_total,PAR_mean,PAR_std,UVA_total,UVA_mean,UVA_std,UVB_total,UVB_mean,UVB_std,UV_idx_mean,UV_idx_median,UV_idx_std,gwet_top_mean,gwet_top_median,gwet_top_std,gwet_top_95p,gwet_top_5p,gwet_root_mean,gwet_root_median,gwet_root_std,gwet_root_95p,gwet_root_5p,gwet_prof_mean,gwet_prof_median,gwet_prof_std,gwet_prof_95p,gwet_prof_5p,PAR_x_gwet_root_mean,PAR_x_gwet_root_std,UVB_div_gwet_top_mean,UVB_div_gwet_top_std,max_consecutive_dry_days,gwet_gradient_mean,gwet_gradient_std,UVA_UVB_ratio_mean,UVA_UVB_ratio_median,UVA_UVB_ratio_std,PAR_fraction_mean,PAR_fraction_median,PAR_fraction_std,dry_days_pct,high_UV_days_pct,C3_PAR_active_days_pct,C4_PAR_active_days_pct,temp_max_mean,temp_max_median,temp_max_std,temp_max_95p,temp_max_above_30_pct,temp_max_above_35_pct,temp_min_mean,temp_min_median,temp_min_std,temp_min_5p,temp_min_below_10_pct,temp_min_below_5_pct,temp_range_mean,temp_range_median,temp_range_std,temp_range_above_10_pct,temp_range_above_15_pct,temp_range_above_20_pct,gdd_total,gdd_mean_daily,rh_max_mean,rh_max_std,rh_min_mean,rh_min_std,rh_range_mean,rh_range_std,high_humidity_pct,low_humidity_pct,vpd_upper_mean,vpd_lower_mean,vpd_middle_mean,vpd_stressed_pct,vpd_recovery_risk_pct,total_rainfall,rainfall_mean,rainy_days_pct,heavy_rain_days_pct,max_consecutive_no_rain_days,rain_intensity_index,thi_mean,thi_std,thi_95p,thi_5p,thi_mild_pct,thi_moderate_pct,wind_speed_mean,wind_speed_std,calm_days_pct,S_SE_pct,N_NW_pct,W_NW_pct,S_SW_pct,N_NE_pct
0,Bogura,2021S1,Potato,NR,CL,12.972987,54.723058,0.070000,8.678886,3.200000,32.512332,1.940000,5.246790,508.03,5.291979,1.735873,58.64,0.610833,0.227738,1.26,0.013125,0.005678,0.756562,0.745,0.316041,0.769479,0.77,0.088191,0.91,0.6275,0.767917,0.755,0.114652,0.96,0.61,0.743438,0.73,0.108646,0.93,0.59,4.165724,1.786171,0.016798,0.006299,0,0.001563,0.029605,47.071429,44.5,11.987162,0.896114,0.895662,0.008837,0.0,0.0,0.822917,0.677083,27.183567,26.8,4.028707,33.0,0.229167,0.010417,16.145773,15.7,4.20762,10.65,0.010417,0.0,11.037795,12.15,3.103016,0.6875,0.020833,0.0,1119.808315,11.66467,92.912552,4.650572,50.666345,15.777593,42.246207,13.881632,0.96875,0.020833,1.870738,0.137968,1.004353,0.697917,0.0,48.0,0.5,0.020833,0.010417,93,24.0,68.889424,5.8421,79.901313,59.6635,0.239583,0.052083,1.843731,1.13255,0.677,0.135417,0.864583,0.0,0.0,0.0
1,Bogura,2021S1,Potato,NR,CL,12.972987,62.431909,0.060000,9.001190,3.390000,38.311392,1.830000,5.202315,508.03,5.291979,1.735873,58.64,0.610833,0.227738,1.26,0.013125,0.005678,0.756562,0.745,0.316041,0.769479,0.77,0.088191,0.91,0.6275,0.767917,0.755,0.114652,0.96,0.61,0.743438,0.73,0.108646,0.93,0.59,4.165724,1.786171,0.016798,0.006299,0,0.001563,0.029605,47.071429,44.5,11.987162,0.896114,0.895662,0.008837,0.0,0.0,0.822917,0.677083,27.183567,26.8,4.028707,33.0,0.229167,0.010417,16.145773,15.7,4.20762,10.65,0.010417,0.0,11.037795,12.15,3.103016,0.6875,0.020833,0.0,1119.808315,11.66467,92.912552,4.650572,50.666345,15.777593,42.246207,13.881632,0.96875,0.020833,1.870738,0.137968,1.004353,0.697917,0.0,48.0,0.5,0.020833,0.010417,93,24.0,68.889424,5.8421,79.901313,59.6635,0.239583,0.052083,1.843731,1.13255,0.677,0.135417,0.864583,0.0,0.0,0.0
2,Bogura,2021S1,Potato,NR,CL,12.972987,62.396639,0.030000,9.004233,3.290000,36.654217,1.700000,5.355071,508.03,5.291979,1.735873,58.64,0.610833,0.227738,1.26,0.013125,0.005678,0.756562,0.745,0.316041,0.769479,0.77,0.088191,0.91,0.6275,0.767917,0.755,0.114652,0.96,0.61,0.743438,0.73,0.108646,0.93,0.59,4.165724,1.786171,0.016798,0.006299,0,0.001563,0.029605,47.071429,44.5,11.987162,0.896114,0.895662,0.008837,0.0,0.0,0.822917,0.677083,27.183567,26.8,4.028707,33.0,0.229167,0.010417,16.145773,15.7,4.20762,10.65,0.010417,0.0,11.037795,12.15,3.103016,0.6875,0.020833,0.0,1119.808315,11.66467,92.912552,4.650572,50.666345,15.777593,42.246207,13.881632,0.96875,0.020833,1.870738,0.137968,1.004353,0.697917,0.0,48.0,0.5,0.020833,0.010417,93,24.0,68.88942

In [8]:
pd.reset_option('display.max_columns')

In [9]:
df_full.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 110 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    Location                      432 non-null    object 
 1    Season                        432 non-null    object 
 2    Crop                          432 non-null    object 
 3    Residue                       432 non-null    object 
 4    Treatment                     432 non-null    object 
 5    N_in                          432 non-null    float64
 6    N_out                         432 non-null    float64
 7    P_in                          432 non-null    float64
 8    P_out                         432 non-null    float64
 9    K_in                          432 non-null    float64
 10   K_out                         432 non-null    float64
 11   S_in                          432 non-null    float64
 12   S_out                         432 non-null    fl

# 1. Parallel Filtering by Group

In [11]:
# Define control, target columns, and bases for categorical variables
control_cols = ["Location", "Crop", "Residue", "Treatment"]
target_cols   = ["N_in", "N_out", "P_in", "P_out", "K_in", "K_out", "S_in", "S_out"]

baseline_choice = {
    "Location":  "Bogura",
    "Crop":      "Potato",
    "Residue":   "NR",
    "Treatment": "CL",
}

# Order categories with base levels coming first
categories_ordered = {}
for col in control_cols:
    uniq = sorted(df_full[col].unique())
    categories_ordered[col] = [baseline_choice[col]] + [u for u in uniq if u != baseline_choice[col]]

In [12]:
# Split the data into train, validate, and test sets 
# using stratified sampling based on location and crop
df_full["Location_Crop"] = df_full["Location"] + "_" + df_full["Crop"]

train_val_df, test_df = train_test_split(
    df_full, test_size=0.1,
    stratify=df_full["Location_Crop"], random_state=114
)

train_df, val_df = train_test_split(
    train_val_df, test_size=1/9,
    stratify=train_val_df["Location_Crop"], random_state=114
)

In [13]:
# Common functions
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# One-Hot Encoder（fixed category sequence，drop baseline）
ohe = OneHotEncoder(
    categories=[categories_ordered[c] for c in control_cols],
    handle_unknown="ignore",
    drop="first"
)

In [14]:
# Baseline model MO
M0_models, M0_metrics = {}, {}

for y in target_cols:
    preproc = ColumnTransformer(
        [("cat", ohe, control_cols)],
        remainder="drop"
    )
    pipe = Pipeline([("prep", preproc), ("linreg", LinearRegression())])
    pipe.fit(train_df, train_df[y])

    M0_models[y] = pipe
    M0_metrics[y] = {
        "train_R2":   r2_score(train_df[y], pipe.predict(train_df)),
        "train_RMSE": rmse(train_df[y], pipe.predict(train_df)),
        "val_R2":     r2_score(val_df[y],   pipe.predict(val_df)),
        "val_RMSE":   rmse(val_df[y],   pipe.predict(val_df)),
    }

## 1-1 Radiation group `selected_rad`

In [16]:
# Suppress convergence warnings during stability selection
warnings.filterwarnings("ignore", category=ConvergenceWarning)

radiation_cols = [
    "PAR_total", "PAR_mean", "PAR_std", 
    "UVA_total", "UVA_mean", "UVA_std", 
    "UVB_total", "UVB_mean", "UVB_std", 
    "UV_idx_mean", "UV_idx_median", "UV_idx_std", 
    "UVA_UVB_ratio_mean", "UVA_UVB_ratio_median", "UVA_UVB_ratio_std", 
    "PAR_fraction_mean", "PAR_fraction_median", "PAR_fraction_std", 
    "high_UV_days_pct", "C3_PAR_active_days_pct", "C4_PAR_active_days_pct"
]

m = np.round(len(radiation_cols)/2).astype(int)
m

10

In [17]:
# Set scaler
rad_scaler = ColumnTransformer(
    [("rad", StandardScaler(), radiation_cols)],
    remainder="drop"
)

In [18]:
# Stability selection parameters
N_BOOTSTRAPS = 50
MIN_ALPHA_RATIO = 0.5
MAX_ALPHA_RATIO = 3.0
SUBSAMPLE_RATIO = 0.7   # Fraction of data to use in each bootstrap
SCORE_THRESHOLD = 0.1  # Minimum combined score for variable selection

M0_w_rad_models, M0_w_rad_metrics, selected_rad = {}, {}, {}
selection_notes = {}  # Track selection decisions

In [19]:
# Radiation variable selection with stability selection
for y in target_cols:
    # Compute training residuals from baseline model
    resid = train_df[y] - M0_models[y].predict(train_df)
    
    # Prepare radiation features
    X_rad_train = rad_scaler.fit_transform(train_df)
    
    # === Initial alpha estimation with ElasticNetCV ===
    # Use high max_iter and tolerance for initial CV
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        lasso_cv = ElasticNetCV(
            cv=5, 
            l1_ratio=1.0,      # Lasso
            n_alphas=100,
            max_iter=20000,    # Increased for convergence
            tol=1e-3,
            random_state=114
        ).fit(X_rad_train, resid)
    
    base_alpha = lasso_cv.alpha_ * 1.5      # Increase regularization by 50%
    
    # === Enhanced Stability Selection with Hybrid Optimization ===
    n_samples = int(len(resid) * SUBSAMPLE_RATIO)
    selection_counts = np.zeros(len(radiation_cols))
    coef_magnitudes = np.zeros(len(radiation_cols))
    optimizer_used = []     # Track which optimizer was used for diagnostics
    
    for bs_idx in range(N_BOOTSTRAPS):
        # Create bootstrap sample with subsampling
        idx = resample(
            np.arange(len(resid)), 
            n_samples=n_samples,
            replace=False,
            random_state=bs_idx
        )
        X_bs = X_rad_train[idx]
        y_bs = resid.iloc[idx]
        
        # Random alpha in wider range around base alpha
        alpha = base_alpha * np.random.uniform(MIN_ALPHA_RATIO, MAX_ALPHA_RATIO)
        model = None
        
        # Attempt 1: Try standard Lasso with relaxed parameters
        try:
            lasso = ElasticNet(
                alpha=alpha,
                l1_ratio=1.0,
                max_iter=10000,
                tol=1e-3,        # Relaxed tolerance
                random_state=bs_idx
            )
            lasso.fit(X_bs, y_bs)
            model = lasso
            optimizer_used.append("ElasticNet")
            
        except (ConvergenceWarning, LinAlgError):
            # Attempt 2: Switch to LARS algorithm for better convergence
            try:
                lars = LassoLars(
                    alpha=alpha,
                    max_iter=100,      # LARS needs fewer iterations
                    normalize=False,   # Data already standardized
                    random_state=bs_idx
                )
                lars.fit(X_bs, y_bs)
                model = lars
                optimizer_used.append("LassoLars")
                
            except LinAlgError:
                # Final fallback: Weakly regularized Lasso
                lasso_fallback = ElasticNet(
                    alpha=alpha * 0.5,  # Reduce regularization
                    l1_ratio=1.0,
                    max_iter=20000,
                    tol=1e-2,          # Very relaxed tolerance
                    random_state=bs_idx
                )
                lasso_fallback.fit(X_bs, y_bs)
                model = lasso_fallback
                optimizer_used.append("Fallback")
        
        # Identify selected features
        selected_mask = np.abs(model.coef_) > 1e-4
        
        # Update selection counts and coefficient magnitudes
        selection_counts += selected_mask.astype(int)
        coef_magnitudes += np.abs(model.coef_) * selected_mask
    
    # === Selection Probability and Scoring ===
    selection_probs = selection_counts / N_BOOTSTRAPS
    avg_coefs = np.zeros(len(radiation_cols))
    for i in range(len(radiation_cols)):
        if selection_counts[i] > 0:
            avg_coefs[i] = coef_magnitudes[i] / selection_counts[i]
    
    # Create combined stability-impact score
    max_avg_coef = np.max(avg_coefs) if np.max(avg_coefs) > 0 else 1.0
    norm_coefs = avg_coefs / max_avg_coef
    combined_scores = selection_probs * norm_coefs
    
    # === Apply Score Threshold Selection ===
    max_score = np.max(combined_scores)
    note = ""  # Initialize selection note
    
    # Identify variables meeting score threshold
    threshold_idx = [i for i, score in enumerate(combined_scores) if score >= SCORE_THRESHOLD]
    
    if max_score < SCORE_THRESHOLD:
        # Case 1: All scores below threshold - select nothing
        selected_idx = []
        note = f"⚠️ Weak signal (max score={max_score:.4f}<{SCORE_THRESHOLD}) - no variables selected"
    elif len(threshold_idx) > 0:
        # Case 2: Some variables meet threshold - select top m among them
        # Sort threshold-passing indices by score (descending)
        sorted_idx = sorted(threshold_idx, key=lambda i: combined_scores[i], reverse=True)
        selected_idx = sorted_idx[:m]
        
        # Add note if selection was reduced by threshold
        if len(selected_idx) < m:
            note = f"⚠️ Weak signal - only {len(selected_idx)}/{m} variables meet score threshold"
    else:
        # Safety net (shouldn't happen due to max_score check)
        selected_idx = []
        note = "⚠️ Unexpected selection condition - no variables selected"
    
    # Convert to variable names
    selected_vars = [radiation_cols[i] for i in selected_idx]
    selected_rad[y] = selected_vars
    selection_notes[y] = note  # Store note for final output
    
    # === Final model with selected radiation variables ===
    preproc_rad_final = ColumnTransformer(
        [
            ("cat", ohe, control_cols),
            ("rad", StandardScaler(), selected_rad[y])
        ],
        remainder="drop"
    )
    
    pipe_rad_final = Pipeline([
        ("prep", preproc_rad_final),
        ("linreg", LinearRegression())
    ])
    
    pipe_rad_final.fit(train_df, train_df[y])
    M0_w_rad_models[y] = pipe_rad_final

    # Store evaluation metrics and diagnostics
    M0_w_rad_metrics[y] = {
        "train_R2": r2_score(train_df[y], pipe_rad_final.predict(train_df)),
        "train_RMSE": rmse(train_df[y], pipe_rad_final.predict(train_df)),
        "val_R2": r2_score(val_df[y], pipe_rad_final.predict(val_df)),
        "val_RMSE": rmse(val_df[y], pipe_rad_final.predict(val_df)),
        "stability_probs": selection_probs.tolist(),
        "avg_coefs": avg_coefs.tolist(),
        "combined_scores": combined_scores.tolist(),
        "optimizers": optimizer_used,  # Track which optimizers were used
        "max_score": max_score,        # Store max score for reference
        "selection_note": note          # Store selection decision note
    }

In [20]:
# Performance comparison output with notes
pd.set_option("display.max_colwidth", None)

rows_rad = []
for y in target_cols:
    row = {
        "Target": y,
        "M0_val_R2": M0_metrics[y]["val_R2"],
        "M0wRad_val_R2": M0_w_rad_metrics[y]["val_R2"],
        "Num_selected": len(selected_rad[y]),
        "Selected_rad_vars": ", ".join(selected_rad[y]) or "-",
        "Max_score": f"{M0_w_rad_metrics[y]['max_score']:.4f}",
        "Note": selection_notes[y]
    }
    rows_rad.append(row)

print("\n>>> Performance after Radiation-group selection with Score Threshold (validation set R²)")
print(pd.DataFrame(rows_rad).to_string(index=False))


>>> Performance after Radiation-group selection with Score Threshold (validation set R²)
Target  M0_val_R2  M0wRad_val_R2  Num_selected                                                                                                                                                   Selected_rad_vars Max_score                                                      Note
  N_in   0.854356       0.846353             5                                                                                             UVA_UVB_ratio_std, high_UV_days_pct, UVB_std, PAR_std, PAR_fraction_std    0.1600 ⚠️ Weak signal - only 5/10 variables meet score threshold
 N_out   0.731878       0.756909            10           PAR_fraction_median, UV_idx_mean, UVB_total, PAR_fraction_mean, UVA_mean, UVB_std, PAR_std, UVA_UVB_ratio_std, UVA_UVB_ratio_mean, C4_PAR_active_days_pct    1.0000                                                          
  P_in   0.651724       0.645658             2                           

In [21]:
# Diagnostic output: show selection details for each target
print("\n>>> Stability Selection Diagnostics:")
for y in target_cols:
    print(f"\n--- {y} ---")
    print(f"Optimizers used: {', '.join(set(M0_w_rad_metrics[y]['optimizers']))}")
    print(f"Selection note: {M0_w_rad_metrics[y]['selection_note']}")
    
    details = []
    for i, var in enumerate(radiation_cols):
        selected = "✓" if var in selected_rad[y] else ""
        
        details.append({
            "Variable": var,
            "Prob": f"{M0_w_rad_metrics[y]['stability_probs'][i]:.3f}",
            "Avg_coef": f"{M0_w_rad_metrics[y]['avg_coefs'][i]:.4f}",
            "Score": f"{M0_w_rad_metrics[y]['combined_scores'][i]:.5f}",
            "Selected": selected
        })
    details_df = pd.DataFrame(details)
    print(details_df.sort_values("Score", ascending=False).to_string(index=False))


>>> Stability Selection Diagnostics:

--- N_in ---
Optimizers used: ElasticNet
Selection note: ⚠️ Weak signal - only 5/10 variables meet score threshold
              Variable  Prob Avg_coef   Score Selected
     UVA_UVB_ratio_std 0.160   1.0550 0.16000        ✓
      high_UV_days_pct 0.220   0.6284 0.13105        ✓
               UVB_std 0.240   0.5594 0.12726        ✓
               PAR_std 0.200   0.6038 0.11446        ✓
      PAR_fraction_std 0.220   0.4811 0.10032        ✓
C4_PAR_active_days_pct 0.140   0.4632 0.06148         
             PAR_total 0.120   0.4420 0.05027         
               UVA_std 0.100   0.3638 0.03448         
   PAR_fraction_median 0.040   0.6980 0.02646         
C3_PAR_active_days_pct 0.080   0.3330 0.02525         
              UVA_mean 0.020   0.7253 0.01375         
             UVB_total 0.020   0.6534 0.01239         
            UV_idx_std 0.040   0.2589 0.00982         
  UVA_UVB_ratio_median 0.040   0.2511 0.00952         
    UVA_UVB_ratio_mea

In [22]:
# Statsmodels OLS (train set only)
train_for_ols = train_df.copy()
for col in control_cols:
    train_for_ols[col] = pd.Categorical(
        train_for_ols[col],
        categories=categories_ordered[col],
        ordered=True
    )

pd.set_option("display.max_rows", None, "display.max_colwidth", None)
for y in target_cols:
    formula_rhs = " + ".join(
        ["C(Location)", "C(Crop)", "C(Residue)", "C(Treatment)"] + selected_rad[y]
    )
    res = smf.ols(f"{y} ~ {formula_rhs}", data=train_for_ols).fit()

    print(f"\n================  {y} — OLS full results  ================")
    for cat in control_cols:
        print(f"[Baseline] {cat}: {baseline_choice[cat]}")
    print()
    print(res.summary())


================  N_in — OLS full results  ================
[Baseline] Location: Bogura
[Baseline] Crop: Potato
[Baseline] Residue: NR
[Baseline] Treatment: CL

                            OLS Regression Results                            
Dep. Variable:                   N_in   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     192.2
Date:                Tue, 05 Aug 2025   Prob (F-statistic):          2.59e-152
Time:                        23:32:37   Log-Likelihood:                -1559.5
No. Observations:                 344   AIC:                             3151.
Df Residuals:                     328   BIC:                             3212.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                coef    std err 

## 1-2 Gwet (soil moisture) group `selected_gwet`

In [24]:
# Define gwet columns
gwet_cols = [
    "gwet_top_mean", "gwet_top_median", "gwet_top_std", "gwet_top_95p", "gwet_top_5p", 
    "gwet_root_mean", "gwet_root_median", "gwet_root_std", "gwet_root_95p", "gwet_root_5p", 
    "gwet_prof_mean", "gwet_prof_median", "gwet_prof_std", "gwet_prof_95p", "gwet_prof_5p", 
    "gwet_gradient_mean", "gwet_gradient_std", 
    "max_consecutive_dry_days", "dry_days_pct"
]

m_gwet = np.round(len(gwet_cols)/2).astype(int)
m_gwet

10

In [25]:
# Set scaler
gwet_scaler = ColumnTransformer(
    [("gwet", StandardScaler(), gwet_cols)],
    remainder="drop"
)

In [26]:
# Use the same hyper-parameters as for the radiation block
M0_w_gwet_models, M0_w_gwet_metrics, selected_gwet = {}, {}, {}
selection_notes_gwet = {}

# Soil moisture (gwet) variable selection with stability selection
for y in target_cols:
    # 1. Baseline residuals
    resid = train_df[y] - M0_models[y].predict(train_df)

    # 2. Prepare gwet feature matrix
    X_gwet_train = gwet_scaler.fit_transform(train_df)

    # 3. Initial alpha via ElasticNetCV (pure Lasso)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        lasso_cv = ElasticNetCV(
            cv=5,
            l1_ratio=1.0,
            n_alphas=100,
            max_iter=20000,
            tol=1e-3,
            random_state=114
        ).fit(X_gwet_train, resid)

    base_alpha = lasso_cv.alpha_ * 1.5

    # 4. Enhanced stability selection (same bootstrap loop)
    n_samples = int(len(resid) * SUBSAMPLE_RATIO)
    sel_counts = np.zeros(len(gwet_cols))
    coef_mags = np.zeros(len(gwet_cols))
    opt_used = []

    for bs_idx in range(N_BOOTSTRAPS):
        idx = resample(
            np.arange(len(resid)),
            n_samples=n_samples,
            replace=False,
            random_state=bs_idx
        )
        X_bs = X_gwet_train[idx]
        y_bs = resid.iloc[idx]

        alpha = base_alpha * np.random.uniform(MIN_ALPHA_RATIO, MAX_ALPHA_RATIO)
        model = None

        try:
            lasso = ElasticNet(
                alpha=alpha,
                l1_ratio=1.0,
                max_iter=10000,
                tol=1e-3,
                random_state=bs_idx
            )
            lasso.fit(X_bs, y_bs)
            model = lasso
            opt_used.append("ElasticNet")

        except (ConvergenceWarning, LinAlgError):
            try:
                lars = LassoLars(
                    alpha=alpha,
                    max_iter=100,
                    normalize=False,
                    random_state=bs_idx
                )
                lars.fit(X_bs, y_bs)
                model = lars
                opt_used.append("LassoLars")

            except LinAlgError:
                lasso_fb = ElasticNet(
                    alpha=alpha * 0.5,
                    l1_ratio=1.0,
                    max_iter=20000,
                    tol=1e-2,
                    random_state=bs_idx
                )
                lasso_fb.fit(X_bs, y_bs)
                model = lasso_fb
                opt_used.append("Fallback")

        mask = np.abs(model.coef_) > 1e-4
        sel_counts += mask.astype(int)
        coef_mags += np.abs(model.coef_) * mask

    # 5. Combine stability & effect size into scores
    sel_probs = sel_counts / N_BOOTSTRAPS
    avg_coefs = np.divide(coef_mags, sel_counts, out=np.zeros_like(coef_mags), where=sel_counts > 0)
    max_avg_coef = np.max(avg_coefs) if np.max(avg_coefs) > 0 else 1.0
    norm_coefs = avg_coefs / max_avg_coef
    comb_scores = sel_probs * norm_coefs

    # 6. Apply score threshold & choose top-m_gwet
    max_score = np.max(comb_scores)
    note = ""

    thresh_idx = [i for i, s in enumerate(comb_scores) if s >= SCORE_THRESHOLD]

    if max_score < SCORE_THRESHOLD:
        chosen_idx = []
        note = f"⚠️ Weak signal (max score={max_score:.4f}<{SCORE_THRESHOLD}) - no variables selected"
    elif len(thresh_idx) > 0:
        sorted_idx = sorted(thresh_idx, key=lambda i: comb_scores[i], reverse=True)
        chosen_idx = sorted_idx[:m_gwet]
        if len(chosen_idx) < m_gwet:
            note = f"⚠️ Weak signal - only {len(chosen_idx)}/{m_gwet} variables meet score threshold"
    else:
        chosen_idx = []
        note = "⚠️ Unexpected selection condition - no variables selected"

    sel_vars = [gwet_cols[i] for i in chosen_idx]
    selected_gwet[y] = sel_vars
    selection_notes_gwet[y] = note

    # 7. Final model with selected gwet variables
    preproc_gwet_final = ColumnTransformer(
        [
            ("cat", ohe, control_cols),
            ("gwet", StandardScaler(), sel_vars)
        ],
        remainder="drop"
    )

    pipe_gwet_final = Pipeline([
        ("prep", preproc_gwet_final),
        ("linreg", LinearRegression())
    ])
    pipe_gwet_final.fit(train_df, train_df[y])
    M0_w_gwet_models[y] = pipe_gwet_final

    # 8. Store metrics
    M0_w_gwet_metrics[y] = {
        "train_R2": r2_score(train_df[y], pipe_gwet_final.predict(train_df)),
        "train_RMSE": rmse(train_df[y], pipe_gwet_final.predict(train_df)),
        "val_R2": r2_score(val_df[y], pipe_gwet_final.predict(val_df)),
        "val_RMSE": rmse(val_df[y], pipe_gwet_final.predict(val_df)),
        "stability_probs": sel_probs.tolist(),
        "avg_coefs": avg_coefs.tolist(),
        "combined_scores": comb_scores.tolist(),
        "optimizers": opt_used,
        "max_score": max_score,
        "selection_note": note
    }


In [27]:
# Performance comparison output for gwet group
pd.set_option("display.max_colwidth", None)

rows_gwet = []
for y in target_cols:
    rows_gwet.append({
        "Target": y,
        "M0_val_R2": M0_metrics[y]["val_R2"],
        "M0wGwet_val_R2": M0_w_gwet_metrics[y]["val_R2"],
        "Num_selected": len(selected_gwet[y]),
        "Selected_gwet_vars": ", ".join(selected_gwet[y]) or "-",
        "Max_score": f"{M0_w_gwet_metrics[y]['max_score']:.4f}",
        "Note": selection_notes_gwet[y]
    })

print("\n>>> Performance after Gwet-group selection with Score Threshold (validation set R²)")
print(pd.DataFrame(rows_gwet).to_string(index=False))


>>> Performance after Gwet-group selection with Score Threshold (validation set R²)
Target  M0_val_R2  M0wGwet_val_R2  Num_selected                                                                                                                                             Selected_gwet_vars Max_score                                                      Note
  N_in   0.854356        0.853815             2                                                                                                                               gwet_root_std, gwet_gradient_std    0.1514 ⚠️ Weak signal - only 2/10 variables meet score threshold
 N_out   0.731878        0.755754             8                                     gwet_prof_mean, gwet_prof_5p, gwet_top_median, gwet_root_std, gwet_prof_std, gwet_top_mean, gwet_top_std, gwet_prof_median    1.0000 ⚠️ Weak signal - only 8/10 variables meet score threshold
  P_in   0.651724        0.652164             1                                           

In [28]:
# Diagnostics for gwet group
print("\n>>> Stability Selection Diagnostics (gwet):")
for y in target_cols:
    print(f"\n--- {y} ---")
    print(f"Optimizers used: {', '.join(set(M0_w_gwet_metrics[y]['optimizers']))}")
    print(f"Selection note: {M0_w_gwet_metrics[y]['selection_note']}")

    details = []
    for i, var in enumerate(gwet_cols):
        sel = "✓" if var in selected_gwet[y] else ""
        details.append({
            "Variable": var,
            "Prob": f"{M0_w_gwet_metrics[y]['stability_probs'][i]:.3f}",
            "Avg_coef": f"{M0_w_gwet_metrics[y]['avg_coefs'][i]:.4f}",
            "Score": f"{M0_w_gwet_metrics[y]['combined_scores'][i]:.5f}",
            "Selected": sel
        })
    print(pd.DataFrame(details).sort_values("Score", ascending=False).to_string(index=False))


>>> Stability Selection Diagnostics (gwet):

--- N_in ---
Optimizers used: ElasticNet
Selection note: ⚠️ Weak signal - only 2/10 variables meet score threshold
                Variable  Prob Avg_coef   Score Selected
           gwet_root_std 0.160   0.6076 0.15139        ✓
       gwet_gradient_std 0.100   0.6422 0.10000        ✓
        gwet_root_median 0.140   0.4064 0.08861         
      gwet_gradient_mean 0.120   0.3866 0.07223         
           gwet_prof_std 0.160   0.2110 0.05258         
max_consecutive_dry_days 0.100   0.2583 0.04022         
            gwet_top_95p 0.080   0.1637 0.02040         
         gwet_top_median 0.020   0.5150 0.01604         
        gwet_prof_median 0.020   0.4704 0.01465         
           gwet_root_95p 0.060   0.1012 0.00945         
            gwet_root_5p 0.020   0.2811 0.00876         
           gwet_prof_95p 0.020   0.2016 0.00628         
             gwet_top_5p 0.020   0.1628 0.00507         
            gwet_top_std 0.020   0.0299 0

In [29]:
# Statsmodels OLS with gwet variables (train set only)
train_for_ols_gwet = train_df.copy()
for col in control_cols:
    train_for_ols_gwet[col] = pd.Categorical(
        train_for_ols_gwet[col],
        categories=categories_ordered[col],
        ordered=True
    )

pd.set_option("display.max_rows", None, "display.max_colwidth", None)
for y in target_cols:
    rhs = " + ".join(
        ["C(Location)", "C(Crop)", "C(Residue)", "C(Treatment)"] + selected_gwet[y]
    )
    res = smf.ols(f"{y} ~ {rhs}", data=train_for_ols_gwet).fit()

    print(f"\n================  {y} — OLS full results (gwet)  ================")
    for cat in control_cols:
        print(f"[Baseline] {cat}: {baseline_choice[cat]}")
    print()
    print(res.summary())


================  N_in — OLS full results (gwet)  ================
[Baseline] Location: Bogura
[Baseline] Crop: Potato
[Baseline] Residue: NR
[Baseline] Treatment: CL

                            OLS Regression Results                            
Dep. Variable:                   N_in   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     241.4
Date:                Tue, 05 Aug 2025   Prob (F-statistic):          1.34e-155
Time:                        23:32:58   Log-Likelihood:                -1560.1
No. Observations:                 344   AIC:                             3146.
Df Residuals:                     331   BIC:                             3196.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                coef    s

## 1-3 Radwet (radiation x gwet) group `selected_radwet`

In [31]:
# Define radwet columns and target group size
radwet_cols = [
    "PAR_x_gwet_root_mean",
    "PAR_x_gwet_root_std",
    "UVB_div_gwet_top_mean",
    "UVB_div_gwet_top_std"
]

m_radwet = np.round(len(radwet_cols)/2).astype(int)
m_radwet

2

In [32]:
# Set scaler
radwet_scaler = ColumnTransformer(
    [("radwet", StandardScaler(), radwet_cols)],
    remainder="drop"
)

In [33]:
# Radiation × Ground-moisture (radwet) variable selection
M0_w_radwet_models, M0_w_radwet_metrics, selected_radwet = {}, {}, {}
selection_notes_radwet = {}

for y in target_cols:
    # 1. Residuals from baseline
    resid = train_df[y] - M0_models[y].predict(train_df)

    # 2. Feature matrix
    X_radwet_train = radwet_scaler.fit_transform(train_df)

    # 3. Initial alpha search
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        lasso_cv = ElasticNetCV(
            cv=5,
            l1_ratio=1.0,
            n_alphas=100,
            max_iter=20000,
            tol=1e-3,
            random_state=114
        ).fit(X_radwet_train, resid)

    base_alpha = lasso_cv.alpha_ * 1.5

    # 4. Stability selection loop
    n_samples = int(len(resid) * SUBSAMPLE_RATIO)
    sel_counts = np.zeros(len(radwet_cols))
    coef_mags = np.zeros(len(radwet_cols))
    opt_used = []

    for bs_idx in range(N_BOOTSTRAPS):
        idx = resample(
            np.arange(len(resid)),
            n_samples=n_samples,
            replace=False,
            random_state=bs_idx
        )
        X_bs = X_radwet_train[idx]
        y_bs = resid.iloc[idx]

        alpha = base_alpha * np.random.uniform(MIN_ALPHA_RATIO, MAX_ALPHA_RATIO)

        try:
            lasso = ElasticNet(
                alpha=alpha,
                l1_ratio=1.0,
                max_iter=10000,
                tol=1e-3,
                random_state=bs_idx
            )
            lasso.fit(X_bs, y_bs)
            model = lasso
            opt_used.append("ElasticNet")
        except (ConvergenceWarning, LinAlgError):
            try:
                lars = LassoLars(
                    alpha=alpha,
                    max_iter=100,
                    normalize=False,
                    random_state=bs_idx
                )
                lars.fit(X_bs, y_bs)
                model = lars
                opt_used.append("LassoLars")
            except LinAlgError:
                fb = ElasticNet(
                    alpha=alpha * 0.5,
                    l1_ratio=1.0,
                    max_iter=20000,
                    tol=1e-2,
                    random_state=bs_idx
                )
                fb.fit(X_bs, y_bs)
                model = fb
                opt_used.append("Fallback")

        mask = np.abs(model.coef_) > 1e-4
        sel_counts += mask.astype(int)
        coef_mags += np.abs(model.coef_) * mask

    # 5. Scoring
    sel_probs = sel_counts / N_BOOTSTRAPS
    avg_coefs = np.divide(coef_mags, sel_counts, out=np.zeros_like(coef_mags), where=sel_counts > 0)
    max_avg_coef = np.max(avg_coefs) if np.max(avg_coefs) > 0 else 1.0
    comb_scores = sel_probs * (avg_coefs / max_avg_coef)

    # 6. Threshold + top-m_radwet
    max_score = np.max(comb_scores)
    note = ""
    thresh_idx = [i for i, s in enumerate(comb_scores) if s >= SCORE_THRESHOLD]

    if max_score < SCORE_THRESHOLD:
        chosen_idx = []
        note = f"⚠️ Weak signal (max score={max_score:.4f}<{SCORE_THRESHOLD}) - no variables selected"
    elif thresh_idx:
        chosen_idx = sorted(thresh_idx, key=lambda i: comb_scores[i], reverse=True)[:m_radwet]
        if len(chosen_idx) < m_radwet:
            note = f"⚠️ Weak signal - only {len(chosen_idx)}/{m_radwet} variables meet score threshold"
    else:
        chosen_idx = []
        note = "⚠️ Unexpected selection condition - no variables selected"

    sel_vars = [radwet_cols[i] for i in chosen_idx]
    selected_radwet[y] = sel_vars
    selection_notes_radwet[y] = note

    # 7. Final model
    preproc_radwet_final = ColumnTransformer(
        [
            ("cat", ohe, control_cols),
            ("radwet", StandardScaler(), sel_vars)
        ],
        remainder="drop"
    )

    pipe_radwet_final = Pipeline([
        ("prep", preproc_radwet_final),
        ("linreg", LinearRegression())
    ])
    pipe_radwet_final.fit(train_df, train_df[y])
    M0_w_radwet_models[y] = pipe_radwet_final

    # 8. Metrics
    M0_w_radwet_metrics[y] = {
        "train_R2": r2_score(train_df[y], pipe_radwet_final.predict(train_df)),
        "train_RMSE": rmse(train_df[y], pipe_radwet_final.predict(train_df)),
        "val_R2": r2_score(val_df[y], pipe_radwet_final.predict(val_df)),
        "val_RMSE": rmse(val_df[y], pipe_radwet_final.predict(val_df)),
        "stability_probs": sel_probs.tolist(),
        "avg_coefs": avg_coefs.tolist(),
        "combined_scores": comb_scores.tolist(),
        "optimizers": opt_used,
        "max_score": max_score,
        "selection_note": note
    }

In [34]:
# Summary table
pd.set_option("display.max_colwidth", None)

rows_radwet = []
for y in target_cols:
    rows_radwet.append({
        "Target": y,
        "M0_val_R2": M0_metrics[y]["val_R2"],
        "M0wRadwet_val_R2": M0_w_radwet_metrics[y]["val_R2"],
        "Num_selected": len(selected_radwet[y]),
        "Selected_radwet_vars": ", ".join(selected_radwet[y]) or "-",
        "Max_score": f"{M0_w_radwet_metrics[y]['max_score']:.4f}",
        "Note": selection_notes_radwet[y]
    })

print("\n>>> Performance after Radwet-group selection with Score Threshold (validation set R²)")
print(pd.DataFrame(rows_radwet).to_string(index=False))


>>> Performance after Radwet-group selection with Score Threshold (validation set R²)
Target  M0_val_R2  M0wRadwet_val_R2  Num_selected                        Selected_radwet_vars Max_score                                                          Note
  N_in   0.854356          0.853276             2   PAR_x_gwet_root_std, UVB_div_gwet_top_std    0.7200                                                              
 N_out   0.731878          0.729208             2 UVB_div_gwet_top_std, UVB_div_gwet_top_mean    1.0000                                                              
  P_in   0.651724          0.653216             1                       UVB_div_gwet_top_mean    0.1593      ⚠️ Weak signal - only 1/2 variables meet score threshold
 P_out   0.824608          0.824608             0                                           -    0.0800 ⚠️ Weak signal (max score=0.0800<0.1) - no variables selected
  K_in   0.817720          0.816958             2 UVB_div_gwet_top_mean, PAR_x_gwet

In [35]:
# Diagnostics
print("\n>>> Stability Selection Diagnostics (radwet):")
for y in target_cols:
    print(f"\n--- {y} ---")
    print(f"Optimizers used: {', '.join(set(M0_w_radwet_metrics[y]['optimizers']))}")
    print(f"Selection note: {M0_w_radwet_metrics[y]['selection_note']}")

    details = []
    for i, var in enumerate(radwet_cols):
        sel = "✓" if var in selected_radwet[y] else ""
        details.append({
            "Variable": var,
            "Prob": f"{M0_w_radwet_metrics[y]['stability_probs'][i]:.3f}",
            "Avg_coef": f"{M0_w_radwet_metrics[y]['avg_coefs'][i]:.4f}",
            "Score": f"{M0_w_radwet_metrics[y]['combined_scores'][i]:.5f}",
            "Selected": sel
        })
    print(pd.DataFrame(details).sort_values("Score", ascending=False).to_string(index=False))


>>> Stability Selection Diagnostics (radwet):

--- N_in ---
Optimizers used: ElasticNet
Selection note: 
             Variable  Prob Avg_coef   Score Selected
  PAR_x_gwet_root_std 0.720   0.7545 0.72000        ✓
 UVB_div_gwet_top_std 0.440   0.3875 0.22600        ✓
 PAR_x_gwet_root_mean 0.500   0.3253 0.21556         
UVB_div_gwet_top_mean 0.400   0.2706 0.14347         

--- N_out ---
Optimizers used: ElasticNet
Selection note: 
             Variable  Prob Avg_coef   Score Selected
 UVB_div_gwet_top_std 1.000  15.2038 1.00000        ✓
UVB_div_gwet_top_mean 1.000  14.1689 0.93193        ✓
 PAR_x_gwet_root_mean 1.000   8.0636 0.53037         
  PAR_x_gwet_root_std 1.000   7.7153 0.50746         

--- P_in ---
Optimizers used: ElasticNet
Selection note: ⚠️ Weak signal - only 1/2 variables meet score threshold
             Variable  Prob Avg_coef   Score Selected
UVB_div_gwet_top_mean 0.200   0.2607 0.15928        ✓
  PAR_x_gwet_root_std 0.080   0.3274 0.08000         
 PAR_x_gwet_root_

In [36]:
# Statsmodels OLS with radwet variables
train_for_ols_radwet = train_df.copy()
for col in control_cols:
    train_for_ols_radwet[col] = pd.Categorical(
        train_for_ols_radwet[col],
        categories=categories_ordered[col],
        ordered=True
    )

pd.set_option("display.max_rows", None, "display.max_colwidth", None)
for y in target_cols:
    rhs = " + ".join(
        ["C(Location)", "C(Crop)", "C(Residue)", "C(Treatment)"] + selected_radwet[y]
    )
    res = smf.ols(f"{y} ~ {rhs}", data=train_for_ols_radwet).fit()

    print(f"\n================  {y} — OLS full results (radwet)  ================")
    for cat in control_cols:
        print(f"[Baseline] {cat}: {baseline_choice[cat]}")
    print()
    print(res.summary())


================  N_in — OLS full results (radwet)  ================
[Baseline] Location: Bogura
[Baseline] Crop: Potato
[Baseline] Residue: NR
[Baseline] Treatment: CL

                            OLS Regression Results                            
Dep. Variable:                   N_in   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     241.2
Date:                Tue, 05 Aug 2025   Prob (F-statistic):          1.49e-155
Time:                        23:32:59   Log-Likelihood:                -1560.2
No. Observations:                 344   AIC:                             3146.
Df Residuals:                     331   BIC:                             3196.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                coef   

## 1-4 GDD (growing degree days) group `selected_gdd`

In [38]:
# Define gdd columns and target group size
gdd_cols = ["gdd_total", "gdd_mean_daily"]

m_gdd = np.round(len(gdd_cols)/2).astype(int)
m_gdd

1

In [39]:
# Set scaler
gdd_scaler = ColumnTransformer(
    [("gdd", StandardScaler(), gdd_cols)],
    remainder="drop"
)

In [40]:
# Growing-Degree-Days (GDD) variable selection with stability selection
M0_w_gdd_models, M0_w_gdd_metrics, selected_gdd = {}, {}, {}
selection_notes_gdd = {}

for y in target_cols:
    # 1. Baseline residuals
    resid = train_df[y] - M0_models[y].predict(train_df)

    # 2. GDD feature matrix
    X_gdd_train = gdd_scaler.fit_transform(train_df)

    # 3. Initial alpha via ElasticNetCV (pure Lasso)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        lasso_cv = ElasticNetCV(
            cv=5,
            l1_ratio=1.0,
            n_alphas=100,
            max_iter=20000,
            tol=1e-3,
            random_state=114
        ).fit(X_gdd_train, resid)

    base_alpha = lasso_cv.alpha_ * 1.5

    # 4. Stability selection loop
    n_samples = int(len(resid) * SUBSAMPLE_RATIO)
    sel_counts = np.zeros(len(gdd_cols))
    coef_mags = np.zeros(len(gdd_cols))
    opt_used = []

    for bs_idx in range(N_BOOTSTRAPS):
        idx = resample(
            np.arange(len(resid)),
            n_samples=n_samples,
            replace=False,
            random_state=bs_idx
        )
        X_bs = X_gdd_train[idx]
        y_bs = resid.iloc[idx]

        alpha = base_alpha * np.random.uniform(MIN_ALPHA_RATIO, MAX_ALPHA_RATIO)

        try:
            lasso = ElasticNet(
                alpha=alpha,
                l1_ratio=1.0,
                max_iter=10000,
                tol=1e-3,
                random_state=bs_idx
            )
            lasso.fit(X_bs, y_bs)
            model = lasso
            opt_used.append("ElasticNet")
        except (ConvergenceWarning, LinAlgError):
            try:
                lars = LassoLars(
                    alpha=alpha,
                    max_iter=100,
                    normalize=False,
                    random_state=bs_idx
                )
                lars.fit(X_bs, y_bs)
                model = lars
                opt_used.append("LassoLars")
            except LinAlgError:
                fb = ElasticNet(
                    alpha=alpha * 0.5,
                    l1_ratio=1.0,
                    max_iter=20000,
                    tol=1e-2,
                    random_state=bs_idx
                )
                fb.fit(X_bs, y_bs)
                model = fb
                opt_used.append("Fallback")

        mask = np.abs(model.coef_) > 1e-4
        sel_counts += mask.astype(int)
        coef_mags  += np.abs(model.coef_) * mask

    # 5. Scoring
    sel_probs   = sel_counts / N_BOOTSTRAPS
    avg_coefs   = np.divide(coef_mags, sel_counts, out=np.zeros_like(coef_mags), where=sel_counts > 0)
    max_avg_c   = np.max(avg_coefs) if np.max(avg_coefs) > 0 else 1.0
    comb_scores = sel_probs * (avg_coefs / max_avg_c)

    # 6. Threshold + top-m_gdd selection
    max_score  = np.max(comb_scores)
    note       = ""
    thresh_idx = [i for i, s in enumerate(comb_scores) if s >= SCORE_THRESHOLD]

    if max_score < SCORE_THRESHOLD:
        chosen_idx = []
        note = f"⚠️ Weak signal (max score={max_score:.4f}<{SCORE_THRESHOLD}) - no variables selected"
    elif thresh_idx:
        chosen_idx = sorted(thresh_idx, key=lambda i: comb_scores[i], reverse=True)[:m_gdd]
        if len(chosen_idx) < m_gdd:
            note = f"⚠️ Weak signal - only {len(chosen_idx)}/{m_gdd} variables meet score threshold"
    else:
        chosen_idx = []
        note = "⚠️ Unexpected selection condition - no variables selected"

    sel_vars = [gdd_cols[i] for i in chosen_idx]
    selected_gdd[y] = sel_vars
    selection_notes_gdd[y] = note

    # 7. Final model with selected GDD variables
    preproc_gdd_final = ColumnTransformer(
        [
            ("cat", ohe, control_cols),
            ("gdd", StandardScaler(), sel_vars)
        ],
        remainder="drop"
    )

    pipe_gdd_final = Pipeline([
        ("prep",  preproc_gdd_final),
        ("linreg", LinearRegression())
    ])
    pipe_gdd_final.fit(train_df, train_df[y])
    M0_w_gdd_models[y] = pipe_gdd_final

    # 8. Metrics
    M0_w_gdd_metrics[y] = {
        "train_R2":  r2_score(train_df[y], pipe_gdd_final.predict(train_df)),
        "train_RMSE": rmse(train_df[y], pipe_gdd_final.predict(train_df)),
        "val_R2":    r2_score(val_df[y], pipe_gdd_final.predict(val_df)),
        "val_RMSE":  rmse(val_df[y], pipe_gdd_final.predict(val_df)),
        "stability_probs": sel_probs.tolist(),
        "avg_coefs":       avg_coefs.tolist(),
        "combined_scores": comb_scores.tolist(),
        "optimizers":      opt_used,
        "max_score":       max_score,
        "selection_note":  note
    }

In [41]:
# Performance comparison output for GDD group
pd.set_option("display.max_colwidth", None)

rows_gdd = []
for y in target_cols:
    rows_gdd.append({
        "Target": y,
        "M0_val_R2":      M0_metrics[y]["val_R2"],
        "M0wGDD_val_R2":  M0_w_gdd_metrics[y]["val_R2"],
        "Num_selected":   len(selected_gdd[y]),
        "Selected_gdd_vars": ", ".join(selected_gdd[y]) or "-",
        "Max_score":      f"{M0_w_gdd_metrics[y]['max_score']:.4f}",
        "Note":           selection_notes_gdd[y]
    })

print("\n>>> Performance after GDD-group selection with Score Threshold (validation set R²)")
print(pd.DataFrame(rows_gdd).to_string(index=False))


>>> Performance after GDD-group selection with Score Threshold (validation set R²)
Target  M0_val_R2  M0wGDD_val_R2  Num_selected Selected_gdd_vars Max_score Note
  N_in   0.854356       0.848606             1         gdd_total    0.5400     
 N_out   0.731878       0.722400             1         gdd_total    0.5644     
  P_in   0.651724       0.653936             1         gdd_total    0.8000     
 P_out   0.824608       0.824039             1         gdd_total    0.7200     
  K_in   0.817720       0.816753             1    gdd_mean_daily    0.7000     
 K_out   0.619455       0.620190             1    gdd_mean_daily    0.9800     
  S_in   0.835257       0.795436             1         gdd_total    0.1000     
 S_out   0.810077       0.802406             1    gdd_mean_daily    0.3000     


In [42]:
# Diagnostics
print("\n>>> Stability Selection Diagnostics (GDD):")
for y in target_cols:
    print(f"\n--- {y} ---")
    print(f"Optimizers used: {', '.join(set(M0_w_gdd_metrics[y]['optimizers']))}")
    print(f"Selection note:  {M0_w_gdd_metrics[y]['selection_note']}")

    details = []
    for i, var in enumerate(gdd_cols):
        sel = "✓" if var in selected_gdd[y] else ""
        details.append({
            "Variable": var,
            "Prob":     f"{M0_w_gdd_metrics[y]['stability_probs'][i]:.3f}",
            "Avg_coef": f"{M0_w_gdd_metrics[y]['avg_coefs'][i]:.4f}",
            "Score":    f"{M0_w_gdd_metrics[y]['combined_scores'][i]:.5f}",
            "Selected": sel
        })
    print(pd.DataFrame(details).sort_values("Score", ascending=False).to_string(index=False))


>>> Stability Selection Diagnostics (GDD):

--- N_in ---
Optimizers used: ElasticNet
Selection note:  
      Variable  Prob Avg_coef   Score Selected
     gdd_total 0.540   0.7967 0.54000        ✓
gdd_mean_daily 0.320   0.5200 0.20887         

--- N_out ---
Optimizers used: ElasticNet
Selection note:  
      Variable  Prob Avg_coef   Score Selected
     gdd_total 0.600   0.4135 0.56443        ✓
gdd_mean_daily 0.400   0.4395 0.40000         

--- P_in ---
Optimizers used: ElasticNet
Selection note:  
      Variable  Prob Avg_coef   Score Selected
     gdd_total 0.800   0.3854 0.80000        ✓
gdd_mean_daily 0.760   0.3524 0.69490         

--- P_out ---
Optimizers used: ElasticNet
Selection note:  
      Variable  Prob Avg_coef   Score Selected
     gdd_total 0.720   0.1034 0.72000        ✓
gdd_mean_daily 0.680   0.0865 0.56871         

--- K_in ---
Optimizers used: ElasticNet
Selection note:  
      Variable  Prob Avg_coef   Score Selected
gdd_mean_daily 0.700   0.6779 0.70000      

In [43]:
# Statsmodels OLS with GDD variables
train_for_ols_gdd = train_df.copy()
for col in control_cols:
    train_for_ols_gdd[col] = pd.Categorical(
        train_for_ols_gdd[col],
        categories=categories_ordered[col],
        ordered=True
    )

pd.set_option("display.max_rows", None, "display.max_colwidth", None)
for y in target_cols:
    rhs = " + ".join(
        ["C(Location)", "C(Crop)", "C(Residue)", "C(Treatment)"] + selected_gdd[y]
    )
    res = smf.ols(f"{y} ~ {rhs}", data=train_for_ols_gdd).fit()

    print(f"\n================  {y} — OLS full results (GDD)  ================")
    for cat in control_cols:
        print(f"[Baseline] {cat}: {baseline_choice[cat]}")
    print()
    print(res.summary())



================  N_in — OLS full results (GDD)  ================
[Baseline] Location: Bogura
[Baseline] Crop: Potato
[Baseline] Residue: NR
[Baseline] Treatment: CL

                            OLS Regression Results                            
Dep. Variable:                   N_in   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     265.2
Date:                Tue, 05 Aug 2025   Prob (F-statistic):          4.47e-157
Time:                        23:32:59   Log-Likelihood:                -1559.5
No. Observations:                 344   AIC:                             3143.
Df Residuals:                     332   BIC:                             3189.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                coef    st

## 1-5 Humidity (air moisture) group `selected_hum`

In [45]:
# Define humidity columns and target group size
hum_cols = [
    "rh_max_mean",  "rh_max_std",
    "rh_min_mean",  "rh_min_std",
    "rh_range_mean","rh_range_std",
    "high_humidity_pct", "low_humidity_pct"
]

m_hum = np.round(len(hum_cols)/2).astype(int)
m_hum

4

In [46]:
# Set scaler
hum_scaler = ColumnTransformer(
    [("hum", StandardScaler(), hum_cols)],
    remainder="drop"
)

In [47]:
# Relative-humidity (hum) variable selection with stability selection
M0_w_hum_models, M0_w_hum_metrics, selected_hum = {}, {}, {}
selection_notes_hum = {}

for y in target_cols:
    # 1. Baseline residuals
    resid = train_df[y] - M0_models[y].predict(train_df)

    # 2. Humidity feature matrix
    X_hum_train = hum_scaler.fit_transform(train_df)

    # 3. Initial alpha via ElasticNetCV
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        lasso_cv = ElasticNetCV(
            cv=5,
            l1_ratio=1.0,
            n_alphas=100,
            max_iter=20000,
            tol=1e-3,
            random_state=114
        ).fit(X_hum_train, resid)

    base_alpha = lasso_cv.alpha_ * 1.5

    # 4. Stability selection loop
    n_samples = int(len(resid) * SUBSAMPLE_RATIO)
    sel_counts = np.zeros(len(hum_cols))
    coef_mags = np.zeros(len(hum_cols))
    opt_used = []

    for bs_idx in range(N_BOOTSTRAPS):
        idx = resample(
            np.arange(len(resid)),
            n_samples=n_samples,
            replace=False,
            random_state=bs_idx
        )
        X_bs = X_hum_train[idx]
        y_bs = resid.iloc[idx]

        alpha = base_alpha * np.random.uniform(MIN_ALPHA_RATIO, MAX_ALPHA_RATIO)

        try:
            lasso = ElasticNet(
                alpha=alpha,
                l1_ratio=1.0,
                max_iter=10000,
                tol=1e-3,
                random_state=bs_idx
            )
            lasso.fit(X_bs, y_bs)
            model = lasso
            opt_used.append("ElasticNet")
        except (ConvergenceWarning, LinAlgError):
            try:
                lars = LassoLars(
                    alpha=alpha,
                    max_iter=100,
                    normalize=False,
                    random_state=bs_idx
                )
                lars.fit(X_bs, y_bs)
                model = lars
                opt_used.append("LassoLars")
            except LinAlgError:
                fb = ElasticNet(
                    alpha=alpha * 0.5,
                    l1_ratio=1.0,
                    max_iter=20000,
                    tol=1e-2,
                    random_state=bs_idx
                )
                fb.fit(X_bs, y_bs)
                model = fb
                opt_used.append("Fallback")

        mask = np.abs(model.coef_) > 1e-4
        sel_counts   += mask.astype(int)
        coef_mags    += np.abs(model.coef_) * mask

    # 5. Scoring
    sel_probs  = sel_counts / N_BOOTSTRAPS
    avg_coefs  = np.divide(coef_mags, sel_counts, out=np.zeros_like(coef_mags), where=sel_counts > 0)
    max_avg_c  = np.max(avg_coefs) if np.max(avg_coefs) > 0 else 1.0
    comb_scores = sel_probs * (avg_coefs / max_avg_c)

    # 6. Threshold + top-m_hum
    max_score = np.max(comb_scores)
    note = ""
    thresh_idx = [i for i, s in enumerate(comb_scores) if s >= SCORE_THRESHOLD]

    if max_score < SCORE_THRESHOLD:
        chosen_idx = []
        note = f"⚠️ Weak signal (max score={max_score:.4f}<{SCORE_THRESHOLD}) - no variables selected"
    elif thresh_idx:
        chosen_idx = sorted(thresh_idx, key=lambda i: comb_scores[i], reverse=True)[:m_hum]
        if len(chosen_idx) < m_hum:
            note = f"⚠️ Weak signal - only {len(chosen_idx)}/{m_hum} variables meet score threshold"
    else:
        chosen_idx = []
        note = "⚠️ Unexpected selection condition - no variables selected"

    sel_vars = [hum_cols[i] for i in chosen_idx]
    selected_hum[y] = sel_vars
    selection_notes_hum[y] = note

    # 7. Final model with selected humidity variables
    preproc_hum_final = ColumnTransformer(
        [
            ("cat",  ohe, control_cols),
            ("hum",  StandardScaler(), sel_vars)
        ],
        remainder="drop"
    )
    pipe_hum_final = Pipeline([
        ("prep",  preproc_hum_final),
        ("linreg", LinearRegression())
    ])
    pipe_hum_final.fit(train_df, train_df[y])
    M0_w_hum_models[y] = pipe_hum_final

    # 8. Metrics
    M0_w_hum_metrics[y] = {
        "train_R2":  r2_score(train_df[y], pipe_hum_final.predict(train_df)),
        "train_RMSE": rmse(train_df[y], pipe_hum_final.predict(train_df)),
        "val_R2":    r2_score(val_df[y], pipe_hum_final.predict(val_df)),
        "val_RMSE":  rmse(val_df[y], pipe_hum_final.predict(val_df)),
        "stability_probs": sel_probs.tolist(),
        "avg_coefs":       avg_coefs.tolist(),
        "combined_scores": comb_scores.tolist(),
        "optimizers":      opt_used,
        "max_score":       max_score,
        "selection_note":  note
    }

In [48]:
# Performance comparison output for humidity group
pd.set_option("display.max_colwidth", None)

rows_hum = []
for y in target_cols:
    rows_hum.append({
        "Target": y,
        "M0_val_R2":      M0_metrics[y]["val_R2"],
        "M0wHum_val_R2":  M0_w_hum_metrics[y]["val_R2"],
        "Num_selected":   len(selected_hum[y]),
        "Selected_hum_vars": ", ".join(selected_hum[y]) or "-",
        "Max_score":      f"{M0_w_hum_metrics[y]['max_score']:.4f}",
        "Note":           selection_notes_hum[y]
    })

print("\n>>> Performance after Humidity-group selection with Score Threshold (validation set R²)")
print(pd.DataFrame(rows_hum).to_string(index=False))



>>> Performance after Humidity-group selection with Score Threshold (validation set R²)
Target  M0_val_R2  M0wHum_val_R2  Num_selected                                              Selected_hum_vars Max_score                                                          Note
  N_in   0.854356       0.852244             2                                        rh_max_std, rh_min_mean    0.2000      ⚠️ Weak signal - only 2/4 variables meet score threshold
 N_out   0.731878       0.738719             2                                  rh_max_std, high_humidity_pct    0.1767      ⚠️ Weak signal - only 2/4 variables meet score threshold
  P_in   0.651724       0.651724             0                                                              -    0.0600 ⚠️ Weak signal (max score=0.0600<0.1) - no variables selected
 P_out   0.824608       0.834579             4  high_humidity_pct, low_humidity_pct, rh_max_std, rh_range_std    0.7400                                                              
 

In [49]:
# Diagnostics
print("\n>>> Stability Selection Diagnostics (humidity):")
for y in target_cols:
    print(f"\n--- {y} ---")
    print(f"Optimizers used: {', '.join(set(M0_w_hum_metrics[y]['optimizers']))}")
    print(f"Selection note:  {M0_w_hum_metrics[y]['selection_note']}")

    details = []
    for i, var in enumerate(hum_cols):
        sel = "✓" if var in selected_hum[y] else ""
        details.append({
            "Variable": var,
            "Prob":     f"{M0_w_hum_metrics[y]['stability_probs'][i]:.3f}",
            "Avg_coef": f"{M0_w_hum_metrics[y]['avg_coefs'][i]:.4f}",
            "Score":    f"{M0_w_hum_metrics[y]['combined_scores'][i]:.5f}",
            "Selected": sel
        })
    print(pd.DataFrame(details).sort_values("Score", ascending=False).to_string(index=False))


>>> Stability Selection Diagnostics (humidity):

--- N_in ---
Optimizers used: ElasticNet
Selection note:  ⚠️ Weak signal - only 2/4 variables meet score threshold
         Variable  Prob Avg_coef   Score Selected
       rh_max_std 0.200   0.7641 0.20000        ✓
      rh_min_mean 0.120   0.6548 0.10283        ✓
 low_humidity_pct 0.100   0.6515 0.08526         
      rh_max_mean 0.080   0.6834 0.07154         
    rh_range_mean 0.140   0.2400 0.04396         
     rh_range_std 0.040   0.5068 0.02653         
high_humidity_pct 0.080   0.2173 0.02275         
       rh_min_std 0.060   0.2553 0.02005         

--- N_out ---
Optimizers used: ElasticNet
Selection note:  ⚠️ Weak signal - only 2/4 variables meet score threshold
         Variable  Prob Avg_coef   Score Selected
       rh_max_std 0.380   0.6651 0.17666        ✓
high_humidity_pct 0.280   0.7877 0.15418        ✓
       rh_min_std 0.020   1.4306 0.02000         
     rh_range_std 0.060   0.4590 0.01925         
      rh_max_mean 

In [50]:
# Statsmodels OLS with humidity variables
train_for_ols_hum = train_df.copy()
for col in control_cols:
    train_for_ols_hum[col] = pd.Categorical(
        train_for_ols_hum[col],
        categories=categories_ordered[col],
        ordered=True
    )

pd.set_option("display.max_rows", None, "display.max_colwidth", None)
for y in target_cols:
    rhs = " + ".join(
        ["C(Location)", "C(Crop)", "C(Residue)", "C(Treatment)"] + selected_hum[y]
    )
    res = smf.ols(f"{y} ~ {rhs}", data=train_for_ols_hum).fit()

    print(f"\n================  {y} — OLS full results (humidity)  ================")
    for cat in control_cols:
        print(f"[Baseline] {cat}: {baseline_choice[cat]}")
    print()
    print(res.summary())



================  N_in — OLS full results (humidity)  ================
[Baseline] Location: Bogura
[Baseline] Crop: Potato
[Baseline] Residue: NR
[Baseline] Treatment: CL

                            OLS Regression Results                            
Dep. Variable:                   N_in   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     241.9
Date:                Tue, 05 Aug 2025   Prob (F-statistic):          9.87e-156
Time:                        23:33:00   Log-Likelihood:                -1559.8
No. Observations:                 344   AIC:                             3146.
Df Residuals:                     331   BIC:                             3196.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                coef 

## 1-6 Temperature group `selected_temp`

In [52]:
# Define columns and target group size
temp_cols = [
    "temp_max_mean", "temp_max_median", "temp_max_std", 
    "temp_max_95p", "temp_max_above_30_pct", "temp_max_above_35_pct", 
    "temp_min_mean", "temp_min_median", "temp_min_std", 
    "temp_min_5p", "temp_min_below_10_pct", "temp_min_below_5_pct", 
    "temp_range_mean", "temp_range_median", "temp_range_std", 
    "temp_range_above_10_pct", "temp_range_above_15_pct", "temp_range_above_20_pct"
]

m_temp = np.round(len(temp_cols)/2).astype(int)
m_temp

9

In [53]:
# Set scaler
temp_scaler = ColumnTransformer(
    [("temp", StandardScaler(), temp_cols)],
    remainder="drop"
)

In [54]:
# Temperature variable selection with stability selection
M0_w_temp_models, M0_w_temp_metrics, selected_temp = {}, {}, {}
selection_notes_temp = {}

for y in target_cols:
    # 1. Baseline residuals
    resid = train_df[y] - M0_models[y].predict(train_df)

    # 2. Temperature feature matrix
    X_temp_train = temp_scaler.fit_transform(train_df)

    # 3. Initial alpha via ElasticNetCV
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        lasso_cv = ElasticNetCV(
            cv=5,
            l1_ratio=1.0,
            n_alphas=100,
            max_iter=20000,
            tol=1e-3,
            random_state=114
        ).fit(X_temp_train, resid)

    base_alpha = lasso_cv.alpha_ * 1.5

    # 4. Stability selection loop
    n_samples = int(len(resid) * SUBSAMPLE_RATIO)
    sel_counts = np.zeros(len(temp_cols))
    coef_mags = np.zeros(len(temp_cols))
    opt_used = []

    for bs_idx in range(N_BOOTSTRAPS):
        idx = resample(
            np.arange(len(resid)),
            n_samples=n_samples,
            replace=False,
            random_state=bs_idx
        )
        X_bs = X_temp_train[idx]
        y_bs = resid.iloc[idx]

        alpha = base_alpha * np.random.uniform(MIN_ALPHA_RATIO, MAX_ALPHA_RATIO)

        try:
            lasso = ElasticNet(
                alpha=alpha,
                l1_ratio=1.0,
                max_iter=10000,
                tol=1e-3,
                random_state=bs_idx
            )
            lasso.fit(X_bs, y_bs)
            model = lasso
            opt_used.append("ElasticNet")
        except (ConvergenceWarning, LinAlgError):
            try:
                lars = LassoLars(
                    alpha=alpha,
                    max_iter=100,
                    normalize=False,
                    random_state=bs_idx
                )
                lars.fit(X_bs, y_bs)
                model = lars
                opt_used.append("LassoLars")
            except LinAlgError:
                fb = ElasticNet(
                    alpha=alpha * 0.5,
                    l1_ratio=1.0,
                    max_iter=20000,
                    tol=1e-2,
                    random_state=bs_idx
                )
                fb.fit(X_bs, y_bs)
                model = fb
                opt_used.append("Fallback")

        mask = np.abs(model.coef_) > 1e-4
        sel_counts += mask.astype(int)
        coef_mags  += np.abs(model.coef_) * mask

    # 5. Scoring
    sel_probs  = sel_counts / N_BOOTSTRAPS
    avg_coefs  = np.divide(coef_mags, sel_counts, out=np.zeros_like(coef_mags), where=sel_counts > 0)
    max_avg_c  = np.max(avg_coefs) if np.max(avg_coefs) > 0 else 1.0
    comb_scores = sel_probs * (avg_coefs / max_avg_c)

    # 6. Threshold + top-m_temp
    max_score = np.max(comb_scores)
    note = ""
    thresh_idx = [i for i, s in enumerate(comb_scores) if s >= SCORE_THRESHOLD]

    if max_score < SCORE_THRESHOLD:
        chosen_idx = []
        note = f"⚠️ Weak signal (max score={max_score:.4f}<{SCORE_THRESHOLD}) - no variables selected"
    elif thresh_idx:
        chosen_idx = sorted(thresh_idx, key=lambda i: comb_scores[i], reverse=True)[:m_temp]
        if len(chosen_idx) < m_temp:
            note = f"⚠️ Weak signal - only {len(chosen_idx)}/{m_temp} variables meet score threshold"
    else:
        chosen_idx = []
        note = "⚠️ Unexpected selection condition - no variables selected"

    sel_vars = [temp_cols[i] for i in chosen_idx]
    selected_temp[y] = sel_vars
    selection_notes_temp[y] = note

    # 7. Final model with selected temperature variables
    preproc_temp_final = ColumnTransformer(
        [
            ("cat",  ohe, control_cols),
            ("temp", StandardScaler(), sel_vars)
        ],
        remainder="drop"
    )
    pipe_temp_final = Pipeline([
        ("prep",  preproc_temp_final),
        ("linreg", LinearRegression())
    ])
    pipe_temp_final.fit(train_df, train_df[y])
    M0_w_temp_models[y] = pipe_temp_final

    # 8. Metrics
    M0_w_temp_metrics[y] = {
        "train_R2":  r2_score(train_df[y], pipe_temp_final.predict(train_df)),
        "train_RMSE": rmse(train_df[y], pipe_temp_final.predict(train_df)),
        "val_R2":    r2_score(val_df[y], pipe_temp_final.predict(val_df)),
        "val_RMSE":  rmse(val_df[y], pipe_temp_final.predict(val_df)),
        "stability_probs": sel_probs.tolist(),
        "avg_coefs":       avg_coefs.tolist(),
        "combined_scores": comb_scores.tolist(),
        "optimizers":      opt_used,
        "max_score":       max_score,
        "selection_note":  note
    }

In [55]:
# Performance comparison output for temperature group
pd.set_option("display.max_colwidth", None)

rows_temp = []
for y in target_cols:
    rows_temp.append({
        "Target": y,
        "M0_val_R2":      M0_metrics[y]["val_R2"],
        "M0wTemp_val_R2": M0_w_temp_metrics[y]["val_R2"],
        "Num_selected":   len(selected_temp[y]),
        "Selected_temp_vars": ", ".join(selected_temp[y]) or "-",
        "Max_score":      f"{M0_w_temp_metrics[y]['max_score']:.4f}",
        "Note":           selection_notes_temp[y]
    })

print("\n>>> Performance after Temperature-group selection with Score Threshold (validation set R²)")
print(pd.DataFrame(rows_temp).to_string(index=False))


>>> Performance after Temperature-group selection with Score Threshold (validation set R²)
Target  M0_val_R2  M0wTemp_val_R2  Num_selected                                                                                                                                               Selected_temp_vars Max_score                                                          Note
  N_in   0.854356        0.850545             1                                                                                                                                            temp_min_below_10_pct    0.1400      ⚠️ Weak signal - only 1/9 variables meet score threshold
 N_out   0.731878        0.763149             7                          temp_max_std, temp_range_above_15_pct, temp_max_above_35_pct, temp_min_below_5_pct, temp_range_above_20_pct, temp_max_95p, temp_min_std    0.9800      ⚠️ Weak signal - only 7/9 variables meet score threshold
  P_in   0.651724        0.651724             0                  

In [56]:
# Diagnostics
print("\n>>> Stability Selection Diagnostics (temperature):")
for y in target_cols:
    print(f"\n--- {y} ---")
    print(f"Optimizers used: {', '.join(set(M0_w_temp_metrics[y]['optimizers']))}")
    print(f"Selection note:  {M0_w_temp_metrics[y]['selection_note']}")

    details = []
    for i, var in enumerate(temp_cols):
        sel = "✓" if var in selected_temp[y] else ""
        details.append({
            "Variable": var,
            "Prob":     f"{M0_w_temp_metrics[y]['stability_probs'][i]:.3f}",
            "Avg_coef": f"{M0_w_temp_metrics[y]['avg_coefs'][i]:.4f}",
            "Score":    f"{M0_w_temp_metrics[y]['combined_scores'][i]:.5f}",
            "Selected": sel
        })
    print(pd.DataFrame(details).sort_values("Score", ascending=False).to_string(index=False))


>>> Stability Selection Diagnostics (temperature):

--- N_in ---
Optimizers used: ElasticNet
Selection note:  ⚠️ Weak signal - only 1/9 variables meet score threshold
               Variable  Prob Avg_coef   Score Selected
  temp_min_below_10_pct 0.140   0.5083 0.14000        ✓
         temp_range_std 0.120   0.4032 0.09520         
           temp_max_std 0.080   0.3506 0.05518         
  temp_max_above_35_pct 0.060   0.4147 0.04895         
           temp_max_95p 0.060   0.2696 0.03182         
temp_range_above_15_pct 0.060   0.2476 0.02923         
        temp_range_mean 0.040   0.3057 0.02406         
   temp_min_below_5_pct 0.080   0.1258 0.01980         
temp_range_above_20_pct 0.040   0.0228 0.00179         
temp_range_above_10_pct 0.000   0.0000 0.00000         
      temp_range_median 0.000   0.0000 0.00000         
          temp_max_mean 0.000   0.0000 0.00000         
        temp_max_median 0.000   0.0000 0.00000         
           temp_min_std 0.000   0.0000 0.00000  

In [57]:
# Statsmodels OLS with temperature variables
train_for_ols_temp = train_df.copy()
for col in control_cols:
    train_for_ols_temp[col] = pd.Categorical(
        train_for_ols_temp[col],
        categories=categories_ordered[col],
        ordered=True
    )

pd.set_option("display.max_rows", None, "display.max_colwidth", None)
for y in target_cols:
    rhs = " + ".join(
        ["C(Location)", "C(Crop)", "C(Residue)", "C(Treatment)"] + selected_temp[y]
    )
    res = smf.ols(f"{y} ~ {rhs}", data=train_for_ols_temp).fit()

    print(f"\n================  {y} — OLS full results (temperature)  ================")
    for cat in control_cols:
        print(f"[Baseline] {cat}: {baseline_choice[cat]}")
    print()
    print(res.summary())



================  N_in — OLS full results (temperature)  ================
[Baseline] Location: Bogura
[Baseline] Crop: Potato
[Baseline] Residue: NR
[Baseline] Treatment: CL

                            OLS Regression Results                            
Dep. Variable:                   N_in   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     264.9
Date:                Tue, 05 Aug 2025   Prob (F-statistic):          5.34e-157
Time:                        23:33:11   Log-Likelihood:                -1559.7
No. Observations:                 344   AIC:                             3143.
Df Residuals:                     332   BIC:                             3189.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                co

## 1-7 Rainfall group `selected_rain`

In [59]:
# Define columns and target group size
rain_cols = [
    "total_rainfall", "rainfall_mean", 
    "rainy_days_pct", "heavy_rain_days_pct", 
    "max_consecutive_no_rain_days", "rain_intensity_index"
]

m_rain = np.round(len(rain_cols)/2).astype(int)
m_rain

3

In [60]:
# Set scaler
rain_scaler = ColumnTransformer(
    [("rain", StandardScaler(), rain_cols)],
    remainder="drop"
)

In [61]:
# Rainfall variable selection with stability selection
M0_w_rain_models, M0_w_rain_metrics, selected_rain = {}, {}, {}
selection_notes_rain = {}

for y in target_cols:
    # 1. Baseline residuals
    resid = train_df[y] - M0_models[y].predict(train_df)

    # 2. Rainfall feature matrix
    X_rain_train = rain_scaler.fit_transform(train_df)

    # 3. Initial alpha via ElasticNetCV (pure Lasso)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        lasso_cv = ElasticNetCV(
            cv=5,
            l1_ratio=1.0,
            n_alphas=100,
            max_iter=20000,
            tol=1e-3,
            random_state=114
        ).fit(X_rain_train, resid)

    base_alpha = lasso_cv.alpha_ * 1.5

    # 4. Enhanced stability selection loop
    n_samples = int(len(resid) * SUBSAMPLE_RATIO)
    sel_counts = np.zeros(len(rain_cols))
    coef_mags = np.zeros(len(rain_cols))
    opt_used = []

    for bs_idx in range(N_BOOTSTRAPS):
        idx = resample(
            np.arange(len(resid)),
            n_samples=n_samples,
            replace=False,
            random_state=bs_idx
        )
        X_bs = X_rain_train[idx]
        y_bs = resid.iloc[idx]

        alpha = base_alpha * np.random.uniform(MIN_ALPHA_RATIO, MAX_ALPHA_RATIO)

        try:
            lasso = ElasticNet(
                alpha=alpha,
                l1_ratio=1.0,
                max_iter=10000,
                tol=1e-3,
                random_state=bs_idx
            )
            lasso.fit(X_bs, y_bs)
            model = lasso
            opt_used.append("ElasticNet")
        except (ConvergenceWarning, LinAlgError):
            try:
                lars = LassoLars(
                    alpha=alpha,
                    max_iter=100,
                    normalize=False,
                    random_state=bs_idx
                )
                lars.fit(X_bs, y_bs)
                model = lars
                opt_used.append("LassoLars")
            except LinAlgError:
                fb = ElasticNet(
                    alpha=alpha * 0.5,
                    l1_ratio=1.0,
                    max_iter=20000,
                    tol=1e-2,
                    random_state=bs_idx
                )
                fb.fit(X_bs, y_bs)
                model = fb
                opt_used.append("Fallback")

        mask = np.abs(model.coef_) > 1e-4
        sel_counts += mask.astype(int)
        coef_mags  += np.abs(model.coef_) * mask

    # 5. Scoring
    sel_probs  = sel_counts / N_BOOTSTRAPS
    avg_coefs  = np.divide(coef_mags, sel_counts, out=np.zeros_like(coef_mags), where=sel_counts > 0)
    max_avg_c  = np.max(avg_coefs) if np.max(avg_coefs) > 0 else 1.0
    comb_scores = sel_probs * (avg_coefs / max_avg_c)

    # 6. Threshold + top-m_rain
    max_score = np.max(comb_scores)
    note = ""
    thresh_idx = [i for i, s in enumerate(comb_scores) if s >= SCORE_THRESHOLD]

    if max_score < SCORE_THRESHOLD:
        chosen_idx = []
        note = f"⚠️ Weak signal (max score={max_score:.4f}<{SCORE_THRESHOLD}) - no variables selected"
    elif thresh_idx:
        chosen_idx = sorted(thresh_idx, key=lambda i: comb_scores[i], reverse=True)[:m_rain]
        if len(chosen_idx) < m_rain:
            note = f"⚠️ Weak signal - only {len(chosen_idx)}/{m_rain} variables meet score threshold"
    else:
        chosen_idx = []
        note = "⚠️ Unexpected selection condition - no variables selected"

    sel_vars = [rain_cols[i] for i in chosen_idx]
    selected_rain[y] = sel_vars
    selection_notes_rain[y] = note

    # 7. Final model with selected rainfall variables
    preproc_rain_final = ColumnTransformer(
        [
            ("cat",  ohe, control_cols),
            ("rain", StandardScaler(), sel_vars)
        ],
        remainder="drop"
    )
    pipe_rain_final = Pipeline([
        ("prep",  preproc_rain_final),
        ("linreg", LinearRegression())
    ])
    pipe_rain_final.fit(train_df, train_df[y])
    M0_w_rain_models[y] = pipe_rain_final

    # 8. Metrics
    M0_w_rain_metrics[y] = {
        "train_R2":  r2_score(train_df[y], pipe_rain_final.predict(train_df)),
        "train_RMSE": rmse(train_df[y], pipe_rain_final.predict(train_df)),
        "val_R2":    r2_score(val_df[y], pipe_rain_final.predict(val_df)),
        "val_RMSE":  rmse(val_df[y], pipe_rain_final.predict(val_df)),
        "stability_probs": sel_probs.tolist(),
        "avg_coefs":       avg_coefs.tolist(),
        "combined_scores": comb_scores.tolist(),
        "optimizers":      opt_used,
        "max_score":       max_score,
        "selection_note":  note
    }

In [62]:
# Performance comparison output for rainfall group
pd.set_option("display.max_colwidth", None)

rows_rain = []
for y in target_cols:
    rows_rain.append({
        "Target": y,
        "M0_val_R2":      M0_metrics[y]["val_R2"],
        "M0wRain_val_R2": M0_w_rain_metrics[y]["val_R2"],
        "Num_selected":   len(selected_rain[y]),
        "Selected_rain_vars": ", ".join(selected_rain[y]) or "-",
        "Max_score":      f"{M0_w_rain_metrics[y]['max_score']:.4f}",
        "Note":           selection_notes_rain[y]
    })

print("\n>>> Performance after Rainfall-group selection with Score Threshold (validation set R²)")
print(pd.DataFrame(rows_rain).to_string(index=False))


>>> Performance after Rainfall-group selection with Score Threshold (validation set R²)
Target  M0_val_R2  M0wRain_val_R2  Num_selected                                                      Selected_rain_vars Max_score                                                          Note
  N_in   0.854356        0.854356             0                                                                       -    0.0728 ⚠️ Weak signal (max score=0.0728<0.1) - no variables selected
 N_out   0.731878        0.712081             3 heavy_rain_days_pct, max_consecutive_no_rain_days, rain_intensity_index    0.9000                                                              
  P_in   0.651724        0.647039             1                                                    rain_intensity_index    0.1400      ⚠️ Weak signal - only 1/3 variables meet score threshold
 P_out   0.824608        0.830505             3               heavy_rain_days_pct, rainy_days_pct, rain_intensity_index    1.0000              

In [63]:
# Diagnostics
print("\n>>> Stability Selection Diagnostics (rainfall):")
for y in target_cols:
    print(f"\n--- {y} ---")
    print(f"Optimizers used: {', '.join(set(M0_w_rain_metrics[y]['optimizers']))}")
    print(f"Selection note:  {M0_w_rain_metrics[y]['selection_note']}")

    details = []
    for i, var in enumerate(rain_cols):
        sel = "✓" if var in selected_rain[y] else ""
        details.append({
            "Variable": var,
            "Prob":     f"{M0_w_rain_metrics[y]['stability_probs'][i]:.3f}",
            "Avg_coef": f"{M0_w_rain_metrics[y]['avg_coefs'][i]:.4f}",
            "Score":    f"{M0_w_rain_metrics[y]['combined_scores'][i]:.5f}",
            "Selected": sel
        })
    print(pd.DataFrame(details).sort_values("Score", ascending=False).to_string(index=False))



>>> Stability Selection Diagnostics (rainfall):

--- N_in ---
Optimizers used: ElasticNet
Selection note:  ⚠️ Weak signal (max score=0.0728<0.1) - no variables selected
                    Variable  Prob Avg_coef   Score Selected
max_consecutive_no_rain_days 0.120   0.5823 0.07278         
         heavy_rain_days_pct 0.060   0.9601 0.06000         
              total_rainfall 0.080   0.6722 0.05602         
        rain_intensity_index 0.120   0.1659 0.02073         
              rainy_days_pct 0.020   0.4937 0.01029         
               rainfall_mean 0.000   0.0000 0.00000         

--- N_out ---
Optimizers used: ElasticNet
Selection note:  
                    Variable  Prob Avg_coef   Score Selected
         heavy_rain_days_pct 0.900   3.8840 0.90000        ✓
max_consecutive_no_rain_days 1.000   2.6408 0.67991        ✓
        rain_intensity_index 1.000   2.0057 0.51640        ✓
               rainfall_mean 0.460   3.0451 0.36065         
              rainy_days_pct 0.120   

In [64]:
# Statsmodels OLS with rainfall variables
train_for_ols_rain = train_df.copy()
for col in control_cols:
    train_for_ols_rain[col] = pd.Categorical(
        train_for_ols_rain[col],
        categories=categories_ordered[col],
        ordered=True
    )

pd.set_option("display.max_rows", None, "display.max_colwidth", None)
for y in target_cols:
    rhs = " + ".join(
        ["C(Location)", "C(Crop)", "C(Residue)", "C(Treatment)"] + selected_rain[y]
    )
    res = smf.ols(f"{y} ~ {rhs}", data=train_for_ols_rain).fit()

    print(f"\n================  {y} — OLS full results (rainfall)  ================")
    for cat in control_cols:
        print(f"[Baseline] {cat}: {baseline_choice[cat]}")
    print()
    print(res.summary())



================  N_in — OLS full results (rainfall)  ================
[Baseline] Location: Bogura
[Baseline] Crop: Potato
[Baseline] Residue: NR
[Baseline] Treatment: CL

                            OLS Regression Results                            
Dep. Variable:                   N_in   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     291.2
Date:                Tue, 05 Aug 2025   Prob (F-statistic):          5.26e-158
Time:                        23:33:12   Log-Likelihood:                -1560.3
No. Observations:                 344   AIC:                             3143.
Df Residuals:                     333   BIC:                             3185.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                coef 

## 1-8 VPD (vapor-pressure deficit) group `selected_vpd`

In [66]:
# Define columns and target group size
vpd_cols = [
    "vpd_upper_mean",
    "vpd_lower_mean",
    "vpd_middle_mean", 
    "vpd_stressed_pct",
    "vpd_recovery_risk_pct"
]

m_vpd = np.round(len(vpd_cols)/2).astype(int)
m_vpd

2

In [67]:
# Set scaler
vpd_scaler = ColumnTransformer(
    [("vpd", StandardScaler(), vpd_cols)],
    remainder="drop"
)

In [68]:
# Vapour-pressure-deficit (vpd) variable selection
M0_w_vpd_models, M0_w_vpd_metrics, selected_vpd = {}, {}, {}
selection_notes_vpd = {}

for y in target_cols:
    # 1. Baseline residuals
    resid = train_df[y] - M0_models[y].predict(train_df)

    # 2. Feature matrix
    X_vpd_train = vpd_scaler.fit_transform(train_df)

    # 3. Initial alpha via ElasticNetCV
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        lasso_cv = ElasticNetCV(
            cv=5,
            l1_ratio=1.0,
            n_alphas=100,
            max_iter=20000,
            tol=1e-3,
            random_state=114
        ).fit(X_vpd_train, resid)

    base_alpha = lasso_cv.alpha_ * 1.5

    # 4. Stability selection loop
    n_samples = int(len(resid) * SUBSAMPLE_RATIO)
    sel_counts = np.zeros(len(vpd_cols))
    coef_mags = np.zeros(len(vpd_cols))
    opt_used = []

    for bs_idx in range(N_BOOTSTRAPS):
        idx = resample(
            np.arange(len(resid)),
            n_samples=n_samples,
            replace=False,
            random_state=bs_idx
        )
        X_bs = X_vpd_train[idx]
        y_bs = resid.iloc[idx]

        alpha = base_alpha * np.random.uniform(MIN_ALPHA_RATIO, MAX_ALPHA_RATIO)

        try:
            lasso = ElasticNet(
                alpha=alpha,
                l1_ratio=1.0,
                max_iter=10000,
                tol=1e-3,
                random_state=bs_idx
            )
            lasso.fit(X_bs, y_bs)
            model = lasso
            opt_used.append("ElasticNet")
        except (ConvergenceWarning, LinAlgError):
            try:
                lars = LassoLars(
                    alpha=alpha,
                    max_iter=100,
                    normalize=False,
                    random_state=bs_idx
                )
                lars.fit(X_bs, y_bs)
                model = lars
                opt_used.append("LassoLars")
            except LinAlgError:
                fb = ElasticNet(
                    alpha=alpha * 0.5,
                    l1_ratio=1.0,
                    max_iter=20000,
                    tol=1e-2,
                    random_state=bs_idx
                )
                fb.fit(X_bs, y_bs)
                model = fb
                opt_used.append("Fallback")

        mask = np.abs(model.coef_) > 1e-4
        sel_counts += mask.astype(int)
        coef_mags  += np.abs(model.coef_) * mask

    # 5. Scoring
    sel_probs  = sel_counts / N_BOOTSTRAPS
    avg_coefs  = np.divide(coef_mags, sel_counts, out=np.zeros_like(coef_mags), where=sel_counts > 0)
    max_avg_c  = np.max(avg_coefs) if np.max(avg_coefs) > 0 else 1.0
    comb_scores = sel_probs * (avg_coefs / max_avg_c)

    # 6. Threshold + top-m_vpd
    max_score = np.max(comb_scores)
    note = ""
    thresh_idx = [i for i, s in enumerate(comb_scores) if s >= SCORE_THRESHOLD]

    if max_score < SCORE_THRESHOLD:
        chosen_idx = []
        note = f"⚠️ Weak signal (max score={max_score:.4f}<{SCORE_THRESHOLD}) - no variables selected"
    elif thresh_idx:
        chosen_idx = sorted(thresh_idx, key=lambda i: comb_scores[i], reverse=True)[:m_vpd]
        if len(chosen_idx) < m_vpd:
            note = f"⚠️ Weak signal - only {len(chosen_idx)}/{m_vpd} variables meet score threshold"
    else:
        chosen_idx = []
        note = "⚠️ Unexpected selection condition - no variables selected"

    sel_vars = [vpd_cols[i] for i in chosen_idx]
    selected_vpd[y] = sel_vars
    selection_notes_vpd[y] = note

    # 7. Final model with selected VPD variables
    preproc_vpd_final = ColumnTransformer(
        [
            ("cat",  ohe, control_cols),
            ("vpd",  StandardScaler(), sel_vars)
        ],
        remainder="drop"
    )
    pipe_vpd_final = Pipeline([
        ("prep",  preproc_vpd_final),
        ("linreg", LinearRegression())
    ])
    pipe_vpd_final.fit(train_df, train_df[y])
    M0_w_vpd_models[y] = pipe_vpd_final

    # 8. Metrics
    M0_w_vpd_metrics[y] = {
        "train_R2":  r2_score(train_df[y], pipe_vpd_final.predict(train_df)),
        "train_RMSE": rmse(train_df[y], pipe_vpd_final.predict(train_df)),
        "val_R2":    r2_score(val_df[y], pipe_vpd_final.predict(val_df)),
        "val_RMSE":  rmse(val_df[y], pipe_vpd_final.predict(val_df)),
        "stability_probs": sel_probs.tolist(),
        "avg_coefs":       avg_coefs.tolist(),
        "combined_scores": comb_scores.tolist(),
        "optimizers":      opt_used,
        "max_score":       max_score,
        "selection_note":  note
    }


In [69]:
# Performance comparison output for VPD group
pd.set_option("display.max_colwidth", None)

rows_vpd = []
for y in target_cols:
    rows_vpd.append({
        "Target": y,
        "M0_val_R2":      M0_metrics[y]["val_R2"],
        "M0wVPD_val_R2":  M0_w_vpd_metrics[y]["val_R2"],
        "Num_selected":   len(selected_vpd[y]),
        "Selected_vpd_vars": ", ".join(selected_vpd[y]) or "-",
        "Max_score":      f"{M0_w_vpd_metrics[y]['max_score']:.4f}",
        "Note":           selection_notes_vpd[y]
    })

print("\n>>> Performance after VPD-group selection with Score Threshold (validation set R²)")
print(pd.DataFrame(rows_vpd).to_string(index=False))



>>> Performance after VPD-group selection with Score Threshold (validation set R²)
Target  M0_val_R2  M0wVPD_val_R2  Num_selected                     Selected_vpd_vars Max_score                                                          Note
  N_in   0.854356       0.852232             2 vpd_recovery_risk_pct, vpd_upper_mean    0.2517                                                              
 N_out   0.731878       0.721539             2     vpd_middle_mean, vpd_stressed_pct    0.5400                                                              
  P_in   0.651724       0.651724             0                                     -    0.0322 ⚠️ Weak signal (max score=0.0322<0.1) - no variables selected
 P_out   0.824608       0.824608             0                                     -    0.0400 ⚠️ Weak signal (max score=0.0400<0.1) - no variables selected
  K_in   0.817720       0.814747             2      vpd_stressed_pct, vpd_lower_mean    0.1600                                     

In [70]:
# Diagnostics
print("\n>>> Stability Selection Diagnostics (VPD):")
for y in target_cols:
    print(f"\n--- {y} ---")
    print(f"Optimizers used: {', '.join(set(M0_w_vpd_metrics[y]['optimizers']))}")
    print(f"Selection note:  {M0_w_vpd_metrics[y]['selection_note']}")

    details = []
    for i, var in enumerate(vpd_cols):
        sel = "✓" if var in selected_vpd[y] else ""
        details.append({
            "Variable": var,
            "Prob":     f"{M0_w_vpd_metrics[y]['stability_probs'][i]:.3f}",
            "Avg_coef": f"{M0_w_vpd_metrics[y]['avg_coefs'][i]:.4f}",
            "Score":    f"{M0_w_vpd_metrics[y]['combined_scores'][i]:.5f}",
            "Selected": sel
        })
    print(pd.DataFrame(details).sort_values("Score", ascending=False).to_string(index=False))



>>> Stability Selection Diagnostics (VPD):

--- N_in ---
Optimizers used: ElasticNet
Selection note:  
             Variable  Prob Avg_coef   Score Selected
vpd_recovery_risk_pct 0.320   0.5944 0.25168        ✓
       vpd_upper_mean 0.240   0.7557 0.24000        ✓
     vpd_stressed_pct 0.200   0.5588 0.14789         
      vpd_middle_mean 0.140   0.5830 0.10801         
       vpd_lower_mean 0.240   0.1933 0.06138         

--- N_out ---
Optimizers used: ElasticNet
Selection note:  
             Variable  Prob Avg_coef   Score Selected
      vpd_middle_mean 0.540   2.0527 0.54000        ✓
     vpd_stressed_pct 0.520   1.7237 0.43666        ✓
       vpd_lower_mean 0.720   0.7774 0.27269         
vpd_recovery_risk_pct 0.580   0.9502 0.26849         
       vpd_upper_mean 0.000   0.0000 0.00000         

--- P_in ---
Optimizers used: ElasticNet
Selection note:  ⚠️ Weak signal (max score=0.0322<0.1) - no variables selected
             Variable  Prob Avg_coef   Score Selected
vpd_recovery

In [71]:
# Statsmodels OLS with VPD variables
train_for_ols_vpd = train_df.copy()
for col in control_cols:
    train_for_ols_vpd[col] = pd.Categorical(
        train_for_ols_vpd[col],
        categories=categories_ordered[col],
        ordered=True
    )

pd.set_option("display.max_rows", None, "display.max_colwidth", None)
for y in target_cols:
    rhs = " + ".join(
        ["C(Location)", "C(Crop)", "C(Residue)", "C(Treatment)"] + selected_vpd[y]
    )
    res = smf.ols(f"{y} ~ {rhs}", data=train_for_ols_vpd).fit()

    print(f"\n================  {y} — OLS full results (VPD)  ================")
    for cat in control_cols:
        print(f"[Baseline] {cat}: {baseline_choice[cat]}")
    print()
    print(res.summary())



================  N_in — OLS full results (VPD)  ================
[Baseline] Location: Bogura
[Baseline] Crop: Potato
[Baseline] Residue: NR
[Baseline] Treatment: CL

                            OLS Regression Results                            
Dep. Variable:                   N_in   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     241.8
Date:                Tue, 05 Aug 2025   Prob (F-statistic):          1.06e-155
Time:                        23:33:12   Log-Likelihood:                -1559.9
No. Observations:                 344   AIC:                             3146.
Df Residuals:                     331   BIC:                             3196.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                coef    st

## 1-9 THI (temperatur-humidity index) group `selected_thi`

In [73]:
# Define columns and target group size
thi_cols = [
    "thi_mean", "thi_std", "thi_95p", "thi_5p", "thi_mild_pct", "thi_moderate_pct"
]

m_thi = np.round(len(thi_cols)/2).astype(int)
m_thi

3

In [74]:
# Set scaler
thi_scaler = ColumnTransformer(
    [("thi", StandardScaler(), thi_cols)],
    remainder="drop"
)

In [75]:
# Temperature–Humidity Index (THI) variable selection
M0_w_thi_models, M0_w_thi_metrics, selected_thi = {}, {}, {}
selection_notes_thi = {}

for y in target_cols:
    # 1. Baseline residuals
    resid = train_df[y] - M0_models[y].predict(train_df)

    # 2. THI feature matrix
    X_thi_train = thi_scaler.fit_transform(train_df)

    # 3. Initial alpha via ElasticNetCV (pure Lasso)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        lasso_cv = ElasticNetCV(
            cv=5,
            l1_ratio=1.0,
            n_alphas=100,
            max_iter=20000,
            tol=1e-3,
            random_state=114
        ).fit(X_thi_train, resid)

    base_alpha = lasso_cv.alpha_ * 1.5

    # 4. Stability selection loop
    n_samples  = int(len(resid) * SUBSAMPLE_RATIO)
    sel_counts = np.zeros(len(thi_cols))
    coef_mags  = np.zeros(len(thi_cols))
    opt_used   = []

    for bs_idx in range(N_BOOTSTRAPS):
        idx = resample(
            np.arange(len(resid)),
            n_samples=n_samples,
            replace=False,
            random_state=bs_idx
        )
        X_bs = X_thi_train[idx]
        y_bs = resid.iloc[idx]

        alpha = base_alpha * np.random.uniform(MIN_ALPHA_RATIO, MAX_ALPHA_RATIO)

        try:
            lasso = ElasticNet(
                alpha=alpha,
                l1_ratio=1.0,
                max_iter=10000,
                tol=1e-3,
                random_state=bs_idx
            )
            lasso.fit(X_bs, y_bs)
            model = lasso
            opt_used.append("ElasticNet")
        except (ConvergenceWarning, LinAlgError):
            try:
                lars = LassoLars(
                    alpha=alpha,
                    max_iter=100,
                    normalize=False,
                    random_state=bs_idx
                )
                lars.fit(X_bs, y_bs)
                model = lars
                opt_used.append("LassoLars")
            except LinAlgError:
                fb = ElasticNet(
                    alpha=alpha * 0.5,
                    l1_ratio=1.0,
                    max_iter=20000,
                    tol=1e-2,
                    random_state=bs_idx
                )
                fb.fit(X_bs, y_bs)
                model = fb
                opt_used.append("Fallback")

        mask = np.abs(model.coef_) > 1e-4
        sel_counts += mask.astype(int)
        coef_mags  += np.abs(model.coef_) * mask

    # 5. Scoring
    sel_probs   = sel_counts / N_BOOTSTRAPS
    avg_coefs   = np.divide(coef_mags, sel_counts, out=np.zeros_like(coef_mags), where=sel_counts > 0)
    max_avg_c   = np.max(avg_coefs) if np.max(avg_coefs) > 0 else 1.0
    comb_scores = sel_probs * (avg_coefs / max_avg_c)

    # 6. Threshold + top-m_thi selection
    max_score  = np.max(comb_scores)
    note       = ""
    thresh_idx = [i for i, s in enumerate(comb_scores) if s >= SCORE_THRESHOLD]

    if max_score < SCORE_THRESHOLD:
        chosen_idx = []
        note = f"⚠️ Weak signal (max score={max_score:.4f}<{SCORE_THRESHOLD}) - no variables selected"
    elif thresh_idx:
        chosen_idx = sorted(thresh_idx, key=lambda i: comb_scores[i], reverse=True)[:m_thi]
        if len(chosen_idx) < m_thi:
            note = f"⚠️ Weak signal - only {len(chosen_idx)}/{m_thi} variables meet score threshold"
    else:
        chosen_idx = []
        note = "⚠️ Unexpected selection condition - no variables selected"

    sel_vars = [thi_cols[i] for i in chosen_idx]
    selected_thi[y] = sel_vars
    selection_notes_thi[y] = note

    # 7. Final model with selected THI variables
    preproc_thi_final = ColumnTransformer(
        [
            ("cat", ohe, control_cols),
            ("thi", StandardScaler(), sel_vars)
        ],
        remainder="drop"
    )

    pipe_thi_final = Pipeline([
        ("prep",  preproc_thi_final),
        ("linreg", LinearRegression())
    ])
    pipe_thi_final.fit(train_df, train_df[y])
    M0_w_thi_models[y] = pipe_thi_final

    # 8. Metrics
    M0_w_thi_metrics[y] = {
        "train_R2":  r2_score(train_df[y], pipe_thi_final.predict(train_df)),
        "train_RMSE": rmse(train_df[y], pipe_thi_final.predict(train_df)),
        "val_R2":    r2_score(val_df[y], pipe_thi_final.predict(val_df)),
        "val_RMSE":  rmse(val_df[y], pipe_thi_final.predict(val_df)),
        "stability_probs": sel_probs.tolist(),
        "avg_coefs":       avg_coefs.tolist(),
        "combined_scores": comb_scores.tolist(),
        "optimizers":      opt_used,
        "max_score":       max_score,
        "selection_note":  note
    }


In [76]:
# Performance comparison output for THI group
pd.set_option("display.max_colwidth", None)

rows_thi = []
for y in target_cols:
    rows_thi.append({
        "Target": y,
        "M0_val_R2":      M0_metrics[y]["val_R2"],
        "M0wTHI_val_R2":  M0_w_thi_metrics[y]["val_R2"],
        "Num_selected":   len(selected_thi[y]),
        "Selected_thi_vars": ", ".join(selected_thi[y]) or "-",
        "Max_score":      f"{M0_w_thi_metrics[y]['max_score']:.4f}",
        "Note":           selection_notes_thi[y]
    })

print("\n>>> Performance after THI-group selection with Score Threshold (validation set R²)")
print(pd.DataFrame(rows_thi).to_string(index=False))



>>> Performance after THI-group selection with Score Threshold (validation set R²)
Target  M0_val_R2  M0wTHI_val_R2  Num_selected                        Selected_thi_vars Max_score                                                          Note
  N_in   0.854356       0.844101             3            thi_mild_pct, thi_95p, thi_5p    0.1759                                                              
 N_out   0.731878       0.742758             3       thi_moderate_pct, thi_mean, thi_5p    1.0000                                                              
  P_in   0.651724       0.651724             0                                        -    0.0600 ⚠️ Weak signal (max score=0.0600<0.1) - no variables selected
 P_out   0.824608       0.846687             3                 thi_5p, thi_std, thi_95p    1.0000                                                              
  K_in   0.817720       0.808145             1                                  thi_95p    0.3647      ⚠️ Weak signa

In [77]:
# Diagnostics
print("\n>>> Stability Selection Diagnostics (THI):")
for y in target_cols:
    print(f"\n--- {y} ---")
    print(f"Optimizers used: {', '.join(set(M0_w_thi_metrics[y]['optimizers']))}")
    print(f"Selection note:  {M0_w_thi_metrics[y]['selection_note']}")

    details = []
    for i, var in enumerate(thi_cols):
        sel = "✓" if var in selected_thi[y] else ""
        details.append({
            "Variable": var,
            "Prob":     f"{M0_w_thi_metrics[y]['stability_probs'][i]:.3f}",
            "Avg_coef": f"{M0_w_thi_metrics[y]['avg_coefs'][i]:.4f}",
            "Score":    f"{M0_w_thi_metrics[y]['combined_scores'][i]:.5f}",
            "Selected": sel
        })
    print(pd.DataFrame(details).sort_values("Score", ascending=False).to_string(index=False))



>>> Stability Selection Diagnostics (THI):

--- N_in ---
Optimizers used: ElasticNet
Selection note:  
        Variable  Prob Avg_coef   Score Selected
    thi_mild_pct 0.240   0.4206 0.17590        ✓
         thi_95p 0.180   0.4115 0.12909        ✓
          thi_5p 0.100   0.5738 0.10000        ✓
         thi_std 0.200   0.2311 0.08055         
        thi_mean 0.060   0.3961 0.04141         
thi_moderate_pct 0.040   0.1206 0.00841         

--- N_out ---
Optimizers used: ElasticNet
Selection note:  
        Variable  Prob Avg_coef   Score Selected
thi_moderate_pct 1.000 150.4043 1.00000        ✓
        thi_mean 1.000 121.6147 0.80859        ✓
          thi_5p 1.000  41.3133 0.27468        ✓
    thi_mild_pct 1.000  38.8696 0.25843         
         thi_std 1.000  30.5110 0.20286         
         thi_95p 1.000  16.4186 0.10916         

--- P_in ---
Optimizers used: ElasticNet
Selection note:  ⚠️ Weak signal (max score=0.0600<0.1) - no variables selected
        Variable  Prob Avg_c

In [78]:
# Statsmodels OLS with THI variables
train_for_ols_thi = train_df.copy()
for col in control_cols:
    train_for_ols_thi[col] = pd.Categorical(
        train_for_ols_thi[col],
        categories=categories_ordered[col],
        ordered=True
    )

pd.set_option("display.max_rows", None, "display.max_colwidth", None)
for y in target_cols:
    rhs = " + ".join(
        ["C(Location)", "C(Crop)", "C(Residue)", "C(Treatment)"] + selected_thi[y]
    )
    res = smf.ols(f"{y} ~ {rhs}", data=train_for_ols_thi).fit()

    print(f"\n================  {y} — OLS full results (THI)  ================")
    for cat in control_cols:
        print(f"[Baseline] {cat}: {baseline_choice[cat]}")
    print()
    print(res.summary())



================  N_in — OLS full results (THI)  ================
[Baseline] Location: Bogura
[Baseline] Crop: Potato
[Baseline] Residue: NR
[Baseline] Treatment: CL

                            OLS Regression Results                            
Dep. Variable:                   N_in   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     223.3
Date:                Tue, 05 Aug 2025   Prob (F-statistic):          9.76e-155
Time:                        23:33:17   Log-Likelihood:                -1559.3
No. Observations:                 344   AIC:                             3147.
Df Residuals:                     330   BIC:                             3200.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                coef    st

## 1-10 Wind group `selected_wind`

In [80]:
# Define columns and target group size
wind_cols = [
    "wind_speed_mean", "wind_speed_std", "calm_days_pct",
    "S_SE_pct", "N_NW_pct", "W_NW_pct", "S_SW_pct", "N_NE_pct"
]

m_wind = np.round(len(wind_cols)/2).astype(int)
m_wind

4

In [81]:
# Set scaler
wind_scaler = ColumnTransformer(
    [("wind", StandardScaler(), wind_cols)],
    remainder="drop"
)

In [82]:
# Wind variable selection with stability selection
M0_w_wind_models, M0_w_wind_metrics, selected_wind = {}, {}, {}
selection_notes_wind = {}

for y in target_cols:
    # 1. Baseline residuals
    resid = train_df[y] - M0_models[y].predict(train_df)

    # 2. Wind feature matrix
    X_wind_train = wind_scaler.fit_transform(train_df)

    # 3. Initial alpha via ElasticNetCV (pure Lasso)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        lasso_cv = ElasticNetCV(
            cv=5,
            l1_ratio=1.0,
            n_alphas=100,
            max_iter=20000,
            tol=1e-3,
            random_state=114
        ).fit(X_wind_train, resid)

    base_alpha = lasso_cv.alpha_ * 1.5

    # 4. Stability selection loop
    n_samples = int(len(resid) * SUBSAMPLE_RATIO)
    sel_counts = np.zeros(len(wind_cols))
    coef_mags = np.zeros(len(wind_cols))
    opt_used = []

    for bs_idx in range(N_BOOTSTRAPS):
        idx = resample(
            np.arange(len(resid)),
            n_samples=n_samples,
            replace=False,
            random_state=bs_idx
        )
        X_bs = X_wind_train[idx]
        y_bs = resid.iloc[idx]

        alpha = base_alpha * np.random.uniform(MIN_ALPHA_RATIO, MAX_ALPHA_RATIO)

        try:
            lasso = ElasticNet(
                alpha=alpha,
                l1_ratio=1.0,
                max_iter=10000,
                tol=1e-3,
                random_state=bs_idx
            )
            lasso.fit(X_bs, y_bs)
            model = lasso
            opt_used.append("ElasticNet")
        except (ConvergenceWarning, LinAlgError):
            try:
                lars = LassoLars(
                    alpha=alpha,
                    max_iter=100,
                    normalize=False,
                    random_state=bs_idx
                )
                lars.fit(X_bs, y_bs)
                model = lars
                opt_used.append("LassoLars")
            except LinAlgError:
                fb = ElasticNet(
                    alpha=alpha * 0.5,
                    l1_ratio=1.0,
                    max_iter=20000,
                    tol=1e-2,
                    random_state=bs_idx
                )
                fb.fit(X_bs, y_bs)
                model = fb
                opt_used.append("Fallback")

        mask = np.abs(model.coef_) > 1e-4
        sel_counts += mask.astype(int)
        coef_mags  += np.abs(model.coef_) * mask

    # 5. Scoring
    sel_probs   = sel_counts / N_BOOTSTRAPS
    avg_coefs   = np.divide(coef_mags, sel_counts, out=np.zeros_like(coef_mags), where=sel_counts > 0)
    max_avg_c   = np.max(avg_coefs) if np.max(avg_coefs) > 0 else 1.0
    comb_scores = sel_probs * (avg_coefs / max_avg_c)

    # 6. Threshold + top-m_wind
    max_score  = np.max(comb_scores)
    note       = ""
    thresh_idx = [i for i, s in enumerate(comb_scores) if s >= SCORE_THRESHOLD]

    if max_score < SCORE_THRESHOLD:
        chosen_idx = []
        note = f"⚠️ Weak signal (max score={max_score:.4f}<{SCORE_THRESHOLD}) - no variables selected"
    elif thresh_idx:
        chosen_idx = sorted(thresh_idx, key=lambda i: comb_scores[i], reverse=True)[:m_wind]
        if len(chosen_idx) < m_wind:
            note = f"⚠️ Weak signal - only {len(chosen_idx)}/{m_wind} variables meet score threshold"
    else:
        chosen_idx = []
        note = "⚠️ Unexpected selection condition - no variables selected"

    sel_vars = [wind_cols[i] for i in chosen_idx]
    selected_wind[y] = sel_vars
    selection_notes_wind[y] = note

    # 7. Final model with selected wind variables
    preproc_wind_final = ColumnTransformer(
        [
            ("cat",  ohe, control_cols),
            ("wind", StandardScaler(), sel_vars)
        ],
        remainder="drop"
    )
    pipe_wind_final = Pipeline([
        ("prep",  preproc_wind_final),
        ("linreg", LinearRegression())
    ])
    pipe_wind_final.fit(train_df, train_df[y])
    M0_w_wind_models[y] = pipe_wind_final

    # 8. Metrics
    M0_w_wind_metrics[y] = {
        "train_R2":  r2_score(train_df[y], pipe_wind_final.predict(train_df)),
        "train_RMSE": rmse(train_df[y], pipe_wind_final.predict(train_df)),
        "val_R2":    r2_score(val_df[y], pipe_wind_final.predict(val_df)),
        "val_RMSE":  rmse(val_df[y], pipe_wind_final.predict(val_df)),
        "stability_probs": sel_probs.tolist(),
        "avg_coefs":       avg_coefs.tolist(),
        "combined_scores": comb_scores.tolist(),
        "optimizers":      opt_used,
        "max_score":       max_score,
        "selection_note":  note
    }


In [83]:
# Performance comparison output for wind group
pd.set_option("display.max_colwidth", None)

rows_wind = []
for y in target_cols:
    rows_wind.append({
        "Target": y,
        "M0_val_R2":      M0_metrics[y]["val_R2"],
        "M0wWind_val_R2": M0_w_wind_metrics[y]["val_R2"],
        "Num_selected":   len(selected_wind[y]),
        "Selected_wind_vars": ", ".join(selected_wind[y]) or "-",
        "Max_score":      f"{M0_w_wind_metrics[y]['max_score']:.4f}",
        "Note":           selection_notes_wind[y]
    })

print("\n>>> Performance after Wind-group selection with Score Threshold (validation set R²)")
print(pd.DataFrame(rows_wind).to_string(index=False))



>>> Performance after Wind-group selection with Score Threshold (validation set R²)
Target  M0_val_R2  M0wWind_val_R2  Num_selected                                       Selected_wind_vars Max_score                                                          Note
  N_in   0.854356        0.848730             1                                          wind_speed_mean    0.1600      ⚠️ Weak signal - only 1/4 variables meet score threshold
 N_out   0.731878        0.734148             4                   W_NW_pct, N_NE_pct, N_NW_pct, S_SE_pct    1.0000                                                              
  P_in   0.651724        0.651724             0                                                        -    0.0456 ⚠️ Weak signal (max score=0.0456<0.1) - no variables selected
 P_out   0.824608        0.818356             4 wind_speed_mean, calm_days_pct, S_SE_pct, wind_speed_std    1.0000                                                              
  K_in   0.817720        0.795

In [84]:
# Diagnostics
print("\n>>> Stability Selection Diagnostics (wind):")
for y in target_cols:
    print(f"\n--- {y} ---")
    print(f"Optimizers used: {', '.join(set(M0_w_wind_metrics[y]['optimizers']))}")
    print(f"Selection note:  {M0_w_wind_metrics[y]['selection_note']}")

    details = []
    for i, var in enumerate(wind_cols):
        sel = "✓" if var in selected_wind[y] else ""
        details.append({
            "Variable": var,
            "Prob":     f"{M0_w_wind_metrics[y]['stability_probs'][i]:.3f}",
            "Avg_coef": f"{M0_w_wind_metrics[y]['avg_coefs'][i]:.4f}",
            "Score":    f"{M0_w_wind_metrics[y]['combined_scores'][i]:.5f}",
            "Selected": sel
        })
    print(pd.DataFrame(details).sort_values("Score", ascending=False).to_string(index=False))



>>> Stability Selection Diagnostics (wind):

--- N_in ---
Optimizers used: ElasticNet
Selection note:  ⚠️ Weak signal - only 1/4 variables meet score threshold
       Variable  Prob Avg_coef   Score Selected
wind_speed_mean 0.160   0.9171 0.16000        ✓
 wind_speed_std 0.040   0.3367 0.01469         
       W_NW_pct 0.040   0.1611 0.00702         
       N_NW_pct 0.020   0.3069 0.00669         
       S_SE_pct 0.020   0.1228 0.00268         
  calm_days_pct 0.000   0.0000 0.00000         
       S_SW_pct 0.000   0.0000 0.00000         
       N_NE_pct 0.000   0.0000 0.00000         

--- N_out ---
Optimizers used: ElasticNet
Selection note:  
       Variable  Prob Avg_coef   Score Selected
       W_NW_pct 1.000   3.3032 1.00000        ✓
       N_NE_pct 1.000   2.7308 0.82670        ✓
       N_NW_pct 0.960   1.6455 0.47822        ✓
       S_SE_pct 0.820   0.9712 0.24110        ✓
  calm_days_pct 0.260   0.7465 0.05876         
wind_speed_mean 0.140   1.3254 0.05617         
 wind_spee

In [85]:
# Statsmodels OLS with wind variables
train_for_ols_wind = train_df.copy()
for col in control_cols:
    train_for_ols_wind[col] = pd.Categorical(
        train_for_ols_wind[col],
        categories=categories_ordered[col],
        ordered=True
    )

pd.set_option("display.max_rows", None, "display.max_colwidth", None)
for y in target_cols:
    rhs = " + ".join(
        ["C(Location)", "C(Crop)", "C(Residue)", "C(Treatment)"] + selected_wind[y]
    )
    res = smf.ols(f"{y} ~ {rhs}", data=train_for_ols_wind).fit()

    print(f"\n================  {y} — OLS full results (wind)  ================")
    for cat in control_cols:
        print(f"[Baseline] {cat}: {baseline_choice[cat]}")
    print()
    print(res.summary())



================  N_in — OLS full results (wind)  ================
[Baseline] Location: Bogura
[Baseline] Crop: Potato
[Baseline] Residue: NR
[Baseline] Treatment: CL

                            OLS Regression Results                            
Dep. Variable:                   N_in   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     264.7
Date:                Tue, 05 Aug 2025   Prob (F-statistic):          5.73e-157
Time:                        23:33:18   Log-Likelihood:                -1559.8
No. Observations:                 344   AIC:                             3144.
Df Residuals:                     332   BIC:                             3190.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                coef    s

# 2. Global Optimization

## 2-1 Hierarchical RFE

### 2-1-1 Hierarchical RFE using Linear Regression

In [89]:
# 1. Merge stage-1 selections into `selected_cols`
stage1_dicts = [
    selected_rad, selected_gwet, selected_radwet, selected_gdd,
    selected_hum, selected_temp, selected_rain, selected_vpd,
    selected_thi, selected_wind
]

selected_cols = {
    y: sorted({feat for d in stage1_dicts for feat in d[y]})
    for y in target_cols
}

print("Consolidated selected_cols\n")
for y in target_cols:
    print(f"{y}: {selected_cols[y]}")
    print(f"Totaled {len(selected_cols[y])} variables\n")

Consolidated selected_cols

N_in: ['PAR_fraction_std', 'PAR_std', 'PAR_x_gwet_root_std', 'UVA_UVB_ratio_std', 'UVB_div_gwet_top_std', 'UVB_std', 'gdd_total', 'gwet_gradient_std', 'gwet_root_std', 'high_UV_days_pct', 'rh_max_std', 'rh_min_mean', 'temp_min_below_10_pct', 'thi_5p', 'thi_95p', 'thi_mild_pct', 'vpd_recovery_risk_pct', 'vpd_upper_mean', 'wind_speed_mean']
Totaled 19 variables

N_out: ['C4_PAR_active_days_pct', 'N_NE_pct', 'N_NW_pct', 'PAR_fraction_mean', 'PAR_fraction_median', 'PAR_std', 'S_SE_pct', 'UVA_UVB_ratio_mean', 'UVA_UVB_ratio_std', 'UVA_mean', 'UVB_div_gwet_top_mean', 'UVB_div_gwet_top_std', 'UVB_std', 'UVB_total', 'UV_idx_mean', 'W_NW_pct', 'gdd_total', 'gwet_prof_5p', 'gwet_prof_mean', 'gwet_prof_median', 'gwet_prof_std', 'gwet_root_std', 'gwet_top_mean', 'gwet_top_median', 'gwet_top_std', 'heavy_rain_days_pct', 'high_humidity_pct', 'max_consecutive_no_rain_days', 'rain_intensity_index', 'rh_max_std', 'temp_max_95p', 'temp_max_above_35_pct', 'temp_max_std', 'temp

In [90]:
# 2. Fit M1 (controls + selected_cols) and compare to M0
M1_models, M1_metrics = {}, {}

for y in target_cols:
    numeric_feats = selected_cols[y]              # may be empty
    preproc_M1 = ColumnTransformer(
        [("cat", ohe, control_cols),
         ("num", StandardScaler(), numeric_feats)],
        remainder="drop"
    )

    pipe_M1 = Pipeline([("prep", preproc_M1),
                        ("linreg", LinearRegression())])

    pipe_M1.fit(train_df, train_df[y])
    M1_models[y] = pipe_M1

    M1_metrics[y] = {
        "train_R2": r2_score(train_df[y], pipe_M1.predict(train_df)),
        "val_R2":   r2_score(val_df[y],   pipe_M1.predict(val_df)),
    }

# ── comparison table ───────────────────────────────────────────────────────────
rows_M1 = []
for y in target_cols:
    rows_M1.append({
        "Target": y,
        "M0_val_R2":  M0_metrics[y]["val_R2"],
        "M1_val_R2":  M1_metrics[y]["val_R2"],
        "Δ (M1-M0)":  M1_metrics[y]["val_R2"] - M0_metrics[y]["val_R2"],
        "# extra X":  len(selected_cols[y]),
    })

print("M0 vs M1 on validation set (R²)")
print(pd.DataFrame(rows_M1).to_string(index=False))

M0 vs M1 on validation set (R²)
Target  M0_val_R2  M1_val_R2  Δ (M1-M0)  # extra X
  N_in   0.854356   0.840219  -0.014137         19
 N_out   0.731878   0.757202   0.025324         42
  P_in   0.651724   0.640568  -0.011156          6
 P_out   0.824608   0.844369   0.019761         44
  K_in   0.817720   0.785872  -0.031849         16
 K_out   0.619455   0.786049   0.166594         48
  S_in   0.835257   0.853493   0.018236         47
 S_out   0.810077   0.883283   0.073205         47


In [91]:
# 3. Hierarchical Recursive Feature Elimination (manual, control cols locked)
HIER_RFE_models   = {}
HIER_RFE_metrics  = {}
HIER_RFE_features = {}

for y in target_cols:
    # 3-A. Start with all environment features selected in Stage 1
    cur_feats      = selected_cols[y].copy()
    best_feats     = cur_feats.copy()
    best_val_R2    = M1_metrics[y]["val_R2"]

    while cur_feats:          # stop when no env feature remains
        # 3-B. Fit model with current feature set
        preproc = ColumnTransformer(
            [("cat", ohe, control_cols),
             ("num", StandardScaler(), cur_feats)],
            remainder="drop"
        )
        pipe = Pipeline([("prep", preproc),
                         ("linreg", LinearRegression())])
        pipe.fit(train_df, train_df[y])

        # ── evaluate on validation data
        val_r2 = r2_score(val_df[y], pipe.predict(val_df))

        # ── Track the best subset so far
        if val_r2 >= best_val_R2 - 1e-6:   # tiny tolerance
            best_val_R2 = val_r2
            best_feats  = cur_feats.copy()

        # 3-C. Compute importance (|coef| summed by original feature)
        coef  = pipe.named_steps["linreg"].coef_
        names = pipe.named_steps["prep"].get_feature_names_out()

        # Map each original env feature to aggregate importance
        imp = {f: 0.0 for f in cur_feats}
        for name, c in zip(names, coef):
            if name.startswith("num__"):
                orig = name.split("__")[1]
                imp[orig] += abs(c)

        # Identify least important env feature
        least_imp = min(imp, key=imp.get)
        cur_feats.remove(least_imp)

    # 3-D. Re-fit with the best subset
    final_preproc = ColumnTransformer(
        [("cat", ohe, control_cols),
         ("num", StandardScaler(), best_feats)],
        remainder="drop"
    )
    final_pipe = Pipeline([("prep", final_preproc),
                           ("linreg", LinearRegression())])
    final_pipe.fit(train_df, train_df[y])

    # store results
    HIER_RFE_models[y]   = final_pipe
    HIER_RFE_features[y] = best_feats
    HIER_RFE_metrics[y]  = {
        "train_R2": r2_score(train_df[y], final_pipe.predict(train_df)),
        "val_R2":   r2_score(val_df[y],   final_pipe.predict(val_df)),
    }

# ── summary table for hierarchical RFE ─────────────────────────────────────────
rows_RFE = []
for y in target_cols:
    rows_RFE.append({
        "Target":       y,
        "M1_val_R2":    M1_metrics[y]["val_R2"],
        "RFE_val_R2":   HIER_RFE_metrics[y]["val_R2"],
        "# kept":       len(HIER_RFE_features[y]),
        "Best_feats":   ", ".join(HIER_RFE_features[y]) or "-"
    })

In [92]:
print("Hierarchical RFE results (validation set R²)")
pd.DataFrame(rows_RFE)

Hierarchical RFE results (validation set R²)


,Target,M1_val_R2,RFE_val_R2,# kept,Best_feats
0,N_in,0.840219,0.848606,1,gdd_total
1,N_out,0.757202,0.757574,38,"C4_PAR_active_days_pct, N_NE_pct, N_NW_pct, PAR_fraction_mean, PAR_fraction_median, PAR_std, S_SE_pct, UVA_UVB_ratio_mean, UVA_UVB_ratio_std, UVA_mean, UVB_div_gwet_top_mean, UVB_div_gwet_top_std, UVB_std, UVB_total, UV_idx_mean, W_NW_pct, gdd_total, gwet_prof_5p, gwet_prof_mean, gwet_prof_std, gwet_root_std, gwet_top_mean, gwet_top_std, heavy_rain_days_pct, high_humidity_pct, rain_intensity_index, rh_max_std, temp_max_above_35_pct, temp_max_std, temp_min_below_5_pct, temp_min_std, temp_range_above_15_pct, temp_range_above_20_pct, thi_5p, thi_mean, thi_moderate_pct, vpd_middle_mean, vpd_stressed_pct"
2,P_in,0.640568,0.653936,1,gdd_total
3,P_out,0.844369,0.858049,5,"UVA_total, gwet_root_95p, high_UV_days_pct, temp_max_median, thi_95p"
4,K_in,0.785872,0.816753,1,gdd_mean_daily
5,K_out,0.786049,0.786843,23,"UVA_UVB_ratio_std, UVB_div_gwet_top_mean, UVB_div_gwet_top_std, UVB_std, UVB_total, UV_idx_mean, UV_idx_median, W_NW_pct, calm_days_pct, gwet_root_95p, gwet_top_median, rainfall_mean, rh_max_std, rh_range_mean, temp_max_above_30_pct, temp_max_mean, temp_min_mean, temp_min_std, temp_range_above_10_pct, thi_mean, thi_moderate_pct, vpd_stressed_pct, wind_speed_mean"
6,S_in,0.853493,0.854328,10,"UVA_UVB_ratio_std, UVB_div_gwet_top_mean, W_NW_pct, gwet_gradient_mean, gwet_prof_std, high_humidity_pct, max_consecutive_no_rain_days, temp_max_median, temp_min_std, thi_moderate_pct"
7,S_out,0.883283,0.883464,37,"C4_PAR_active_days_pct, PAR_std, UVA_UVB_ratio_mean, UVA_UVB_ratio_median, UVA_UVB_ratio_std, UVA_total, UVB_div_gwet_top_mean, UVB_div_gwet_top_std, UVB_total, UV_idx_mean, UV_idx_median, W_NW_pct, gwet_prof_5p, gwet_prof_95p, gwet_prof_mean, gwet_top_5p, gwet_top_std, low_humidity_pct, max_consecutive_no_rain_days, rainfall_mean, rainy_days_pct, rh_max_std, rh_min_std, rh_range_std, temp_max_above_30_pct, temp_max_mean, temp_max_median, temp_max_std, temp_min_median, temp_range_above_15_pct, temp_range_mean, temp_range_median, thi_mean, thi_mild_pct, thi_moderate_pct, vpd_stressed_pct, vpd_upper_mean"


### 2-1-2 Hierarchical RFE using Lasso

In [94]:
# === 0. (Optional) Baseline with Lasso on selected cols (analogous to M1) ===
M1_lasso_models = {}
M1_lasso_metrics = {}

for y in target_cols:
    numeric_feats = selected_cols[y]  # could be empty
    preproc_M1_lasso = ColumnTransformer(
        [("cat", ohe, control_cols),
         ("num", StandardScaler(), numeric_feats)],
        remainder="drop"
    )

    # Use LassoCV to pick alpha automatically (you can adjust cv or alphas if needed)
    pipe_M1_lasso = Pipeline([
        ("prep", preproc_M1_lasso),
        ("lasso", LassoCV(cv=5, random_state=114, n_jobs=-1, max_iter=5000))
    ])

    pipe_M1_lasso.fit(train_df, train_df[y])
    M1_lasso_models[y] = pipe_M1_lasso

    M1_lasso_metrics[y] = {
        "train_R2": r2_score(train_df[y], pipe_M1_lasso.predict(train_df)),
        "val_R2":   r2_score(val_df[y],   pipe_M1_lasso.predict(val_df)),
    }


# === Hierarchical RFE with Lasso (control cols locked) ===
HIER_RFE_LASSO_models   = {}
HIER_RFE_LASSO_metrics  = {}
HIER_RFE_LASSO_features = {}

for y in target_cols:
    current_feats  = selected_cols[y].copy()
    best_feats     = current_feats.copy()
    best_val_R2    = M1_lasso_metrics[y]["val_R2"]  # start from Lasso baseline

    while current_feats:
        # Fit with current feature set using LassoCV
        preproc = ColumnTransformer(
            [("cat", ohe, control_cols),
             ("num", StandardScaler(), current_feats)],
            remainder="drop"
        )
        pipe = Pipeline([
            ("prep", preproc),
            ("lasso", LassoCV(cv=5, random_state=114, n_jobs=-1, max_iter=5000))
        ])
        pipe.fit(train_df, train_df[y])

        # evaluate on validation data
        val_r2 = r2_score(val_df[y], pipe.predict(val_df))

        # track best subset (allow ties within tiny tolerance)
        if val_r2 >= best_val_R2 - 1e-6:
            best_val_R2 = val_r2
            best_feats  = current_feats.copy()

        # compute aggregate importance from absolute coefficients
        coef = pipe.named_steps["lasso"].coef_
        names = pipe.named_steps["prep"].get_feature_names_out()

        imp = {f: 0.0 for f in current_feats}
        for name, c in zip(names, coef):
            if name.startswith("num__"):
                orig = name.split("__")[1]
                if orig in imp:
                    imp[orig] += abs(c)

        # if all importances are zero (possible in heavy regularization), break to avoid infinite loop
        if all(v == 0 for v in imp.values()):
            break

        # remove least important feature
        least_imp_feat = min(imp, key=imp.get)
        current_feats.remove(least_imp_feat)

    # Re-fit final model with best_feats
    final_preproc = ColumnTransformer(
        [("cat", ohe, control_cols),
         ("num", StandardScaler(), best_feats)],
        remainder="drop"
    )
    final_pipe = Pipeline([
        ("prep", final_preproc),
        ("lasso", LassoCV(cv=5, random_state=114, n_jobs=-1, max_iter=5000))
    ])
    final_pipe.fit(train_df, train_df[y])

    HIER_RFE_LASSO_models[y]   = final_pipe
    HIER_RFE_LASSO_features[y] = best_feats
    HIER_RFE_LASSO_metrics[y]  = {
        "train_R2": r2_score(train_df[y], final_pipe.predict(train_df)),
        "val_R2":   r2_score(val_df[y],   final_pipe.predict(val_df)),
    }


# === summary table for hierarchical RFE with Lasso ===
rows_RFE_Lasso = []
for y in target_cols:
    rows_RFE_Lasso.append({
        "Target":             y,
        "Lasso_M1_val_R2":    M1_lasso_metrics[y]["val_R2"],
        "Lasso_RFE_val_R2":   HIER_RFE_LASSO_metrics[y]["val_R2"],
        "# kept":             len(HIER_RFE_LASSO_features[y]),
        "Best_feats":         ", ".join(HIER_RFE_LASSO_features[y]) or "-"
    })


In [95]:
print("Hierarchical RFE with Lasso results (validation set R²)")
pd.DataFrame(rows_RFE_Lasso)

Hierarchical RFE with Lasso results (validation set R²)


,Target,Lasso_M1_val_R2,Lasso_RFE_val_R2,# kept,Best_feats
0,N_in,0.844765,0.853112,1,thi_5p
1,N_out,0.745239,0.758032,5,"C4_PAR_active_days_pct, N_NE_pct, N_NW_pct, temp_max_95p, temp_max_std"
2,P_in,0.638058,0.651445,1,gdd_total
3,P_out,0.843211,0.843461,14,"C4_PAR_active_days_pct, S_SE_pct, UVB_std, heavy_rain_days_pct, high_UV_days_pct, rain_intensity_index, rh_max_std, rh_range_std, temp_max_95p, temp_max_median, temp_max_std, temp_range_std, thi_95p, wind_speed_std"
4,K_in,0.791772,0.813632,1,gdd_mean_daily
5,K_out,0.782786,0.789294,9,"N_NE_pct, UVB_div_gwet_top_std, UV_idx_mean, gwet_prof_5p, gwet_root_median, rainfall_mean, rh_max_std, temp_range_median, vpd_stressed_pct"
6,S_in,0.853186,0.878986,4,"UVA_UVB_ratio_std, heavy_rain_days_pct, temp_min_std, temp_range_above_10_pct"
7,S_out,0.881200,0.888209,9,"N_NE_pct, N_NW_pct, PAR_std, gwet_prof_5p, gwet_prof_mean, max_consecutive_no_rain_days, rainfall_mean, rh_max_std, temp_max_above_30_pct"


### Hierarchical RFE using Random Forest

In [97]:
# === 0. Baseline M1 with Random Forest on selected cols ===
M1_rf_models = {}
M1_rf_metrics = {}

for y in target_cols:
    numeric_feats = selected_cols[y]  # could be empty
    preproc_M1_rf = ColumnTransformer(
        [("cat", ohe, control_cols),
         ("num", StandardScaler(), numeric_feats)],  # scaler is optional for RF
        remainder="drop"
    )

    pipe_M1_rf = Pipeline([
        ("prep", preproc_M1_rf),
        ("rf", RandomForestRegressor(n_estimators=500, random_state=114, n_jobs=-1))
    ])

    pipe_M1_rf.fit(train_df, train_df[y])
    M1_rf_models[y] = pipe_M1_rf

    M1_rf_metrics[y] = {
        "train_R2": r2_score(train_df[y], pipe_M1_rf.predict(train_df)),
        "val_R2":   r2_score(val_df[y],   pipe_M1_rf.predict(val_df)),
    }


# === 1. Hierarchical RFE with Random Forest (control cols locked) ===
HIER_RFE_RF_models   = {}
HIER_RFE_RF_metrics  = {}
HIER_RFE_RF_features = {}

for y in target_cols:
    current_feats  = selected_cols[y].copy()
    best_feats     = current_feats.copy()
    best_val_R2    = M1_rf_metrics[y]["val_R2"]  # start from RF baseline

    while current_feats:
        # Fit with current feature set using Random Forest
        preproc = ColumnTransformer(
            [("cat", ohe, control_cols),
             ("num", StandardScaler(), current_feats)],  # scaler optional
            remainder="drop"
        )
        pipe = Pipeline([
            ("prep", preproc),
            ("rf", RandomForestRegressor(n_estimators=500, random_state=114, n_jobs=-1))
        ])
        pipe.fit(train_df, train_df[y])

        # evaluate on validation data
        val_r2 = r2_score(val_df[y], pipe.predict(val_df))

        if val_r2 >= best_val_R2 - 1e-6:
            best_val_R2 = val_r2
            best_feats  = current_feats.copy()

        # compute aggregate importance from RF feature_importances_
        importances = pipe.named_steps["rf"].feature_importances_
        names = pipe.named_steps["prep"].get_feature_names_out()

        imp = {f: 0.0 for f in current_feats}
        for name, imp_val in zip(names, importances):
            if name.startswith("num__"):
                orig = name.split("__")[1]
                if orig in imp:
                    imp[orig] += imp_val

        # if all importances are zero (unlikely), break to avoid infinite loop
        if all(v == 0 for v in imp.values()):
            break

        least_imp_feat = min(imp, key=imp.get)
        current_feats.remove(least_imp_feat)

    # Re-fit final model with best_feats
    final_preproc = ColumnTransformer(
        [("cat", ohe, control_cols),
         ("num", StandardScaler(), best_feats)],
        remainder="drop"
    )
    final_pipe = Pipeline([
        ("prep", final_preproc),
        ("rf", RandomForestRegressor(n_estimators=500, random_state=114, n_jobs=-1))
    ])
    final_pipe.fit(train_df, train_df[y])

    HIER_RFE_RF_models[y]   = final_pipe
    HIER_RFE_RF_features[y] = best_feats
    HIER_RFE_RF_metrics[y]  = {
        "train_R2": r2_score(train_df[y], final_pipe.predict(train_df)),
        "val_R2":   r2_score(val_df[y],   final_pipe.predict(val_df)),
    }


In [98]:
# Summary table for Random Forest hierarchical RFE
rows_RFE_RF = []
for y in target_cols:
    rows_RFE_RF.append({
        "Target":             y,
        "RF_M1_val_R2":       M1_rf_metrics[y]["val_R2"],
        "RF_RFE_val_R2":      HIER_RFE_RF_metrics[y]["val_R2"],
        "# kept":             len(HIER_RFE_RF_features[y]),
        "Best_feats":         ", ".join(HIER_RFE_RF_features[y]) or "-"
    })

print("Hierarchical RFE with Random Forest results (validation set R²)")
pd.DataFrame(rows_RFE_RF)

Hierarchical RFE with Random Forest results (validation set R²)


,Target,RF_M1_val_R2,RF_RFE_val_R2,# kept,Best_feats
0,N_in,0.999743,0.999977,3,"UVA_UVB_ratio_std, gdd_total, thi_5p"
1,N_out,0.953656,0.958504,2,"UVA_UVB_ratio_std, vpd_middle_mean"
2,P_in,0.999927,0.999928,5,"C3_PAR_active_days_pct, C4_PAR_active_days_pct, UVB_div_gwet_top_mean, gdd_total, gwet_root_std"
3,P_out,0.910198,0.915158,1,temp_min_median
4,K_in,0.999931,0.999974,3,"C3_PAR_active_days_pct, gdd_mean_daily, temp_max_median"
5,K_out,0.926423,0.927180,3,"UVA_UVB_ratio_std, temp_max_mean, temp_min_mean"
6,S_in,0.999419,0.999442,32,"PAR_fraction_std, UVA_UVB_ratio_mean, UVA_UVB_ratio_median, UVA_UVB_ratio_std, UVB_div_gwet_top_mean, UVB_std, UVB_total, calm_days_pct, gdd_total, gwet_prof_95p, gwet_prof_mean, gwet_root_95p, gwet_root_mean, gwet_top_5p, gwet_top_95p, gwet_top_median, heavy_rain_days_pct, low_humidity_pct, max_consecutive_no_rain_days, rainy_days_pct, rh_min_mean, rh_range_std, temp_max_median, temp_max_std, temp_min_5p, temp_min_std, temp_range_above_10_pct, temp_range_std, thi_mean, vpd_stressed_pct, vpd_upper_mean, wind_speed_mean"
7,S_out,0.920320,0.922648,5,"gwet_top_median, low_humidity_pct, temp_max_std, temp_range_above_15_pct, vpd_stressed_pct"


## 2-2 Feature clustering elimination based on SHAP values

In [100]:
warnings.filterwarnings("ignore", category=UserWarning)   # suppress SHAP print-outs

# === PARAMETERS ===
N_BOOTSTRAPS         = 10
SUBSAMPLE_FRAC       = 0.80
STABILITY_THRESHOLD  = 8       # co-occur ≥ 8/10 → stable cluster link
SHAP_ZERO_EPS        = 1e-8    # threshold to consider a SHAP vector nearly constant

# === CONTROL COLUMNS ===
control_cols = ["Location", "Crop", "Residue", "Treatment"]

# === STORAGE ===
cluster_kept_feats = {}      # final kept env features per target
cluster_metrics    = {}      # train / val R² after SHAP-cluster selection
cluster_details    = {}      # diagnostic cooccur probabilities
constant_stats     = {}      # how often a feature was (near-)constant in bootstraps

# === MAIN LOOP OVER TARGETS ===
for y in target_cols:
    env_feats = selected_cols[y]
    all_feats = control_cols + env_feats

    # 3-A. Preprocessing to numeric matrix for full feature XGBoost
    cat_ohe = OneHotEncoder(categories=[categories_ordered[c] for c in control_cols],
                            handle_unknown="ignore", drop=None)
    num_scaler = StandardScaler()

    preproc = ColumnTransformer(
        [("cat", cat_ohe, control_cols),
         ("num", num_scaler, env_feats)],
        remainder="drop"
    )

    X_train = preproc.fit_transform(train_df)      # ndarray
    feat_names = preproc.get_feature_names_out()   # flattened feature names
    feat_idx = {f: i for i, f in enumerate(feat_names)}
    n_feat = X_train.shape[1]

    # 3-B. Fit full XGBoost model
    xgb_full = XGBRegressor(
        n_estimators=300, learning_rate=0.05,
        max_depth=4, subsample=0.9, colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=114, n_jobs=-1
    )
    xgb_full.fit(X_train, train_df[y])

    # 3-C. SHAP values (train set)
    explainer = shap.TreeExplainer(xgb_full)
    shap_vals = explainer.shap_values(X_train)    # (n_samples, n_features)
    abs_shap_global = np.mean(np.abs(shap_vals), axis=0)   # global importance

    # 3-D. Bootstrap clustering with safe correlation
    cooccur = np.zeros((n_feat, n_feat), dtype=int)
    constant_counter = np.zeros(n_feat, dtype=int)  # counts of near-constant per bootstrap

    rng = np.random.default_rng(seed=114)
    for b in range(N_BOOTSTRAPS):
        idx = rng.choice(len(train_df), size=int(len(train_df)*SUBSAMPLE_FRAC),
                         replace=False)
        sv = shap_vals[idx]  # (subsample_size, n_feat)

        # compute per-feature mean and std
        means = np.mean(sv, axis=0)
        stds = np.std(sv, axis=0, ddof=0)

        # mark near-constant features
        near_const = stds < SHAP_ZERO_EPS
        constant_counter[near_const] += 1

        # safe pairwise correlation matrix
        corr = np.zeros((n_feat, n_feat), dtype=float)
        for i in range(n_feat):
            for j in range(i, n_feat):
                if stds[i] < SHAP_ZERO_EPS or stds[j] < SHAP_ZERO_EPS:
                    c = 0.0
                else:
                    cov = np.mean((sv[:, i] - means[i]) * (sv[:, j] - means[j]))
                    c = cov / (stds[i] * stds[j])
                    c = max(min(c, 1.0), -1.0)  # clamp numeric drift
                corr[i, j] = c
                corr[j, i] = c

        dist = 1 - np.abs(corr)  # distance for clustering

        # OPTICS clustering on precomputed distance
        optics = OPTICS(metric="precomputed", min_samples=2)
        optics.fit(dist)
        labels = optics.labels_

        # force control columns to be singletons with unique positive labels
        next_label = labels.max() + 1
        for ctrl in control_cols:
            for colname in feat_names:
                if colname.startswith("cat__") and ctrl in colname:
                    labels[feat_idx[colname]] = next_label
                    next_label += 1

        # update co-occurrence counts (same cluster & not noise)
        for i, j in itertools.combinations(range(n_feat), 2):
            if labels[i] == labels[j] and labels[i] != -1:
                cooccur[i, j] += 1
                cooccur[j, i] += 1

    # 3-E. Build stable clusters via union-find from co-occurrence threshold
    parent = list(range(n_feat))
    def find(i):
        while parent[i] != i:
            parent[i] = parent[parent[i]]
            i = parent[i]
        return i
    def union(i, j):
        pi, pj = find(i), find(j)
        if pi != pj:
            parent[pj] = pi

    for i in range(n_feat):
        for j in range(i + 1, n_feat):
            if cooccur[i, j] >= STABILITY_THRESHOLD:
                union(i, j)

    clusters = {}
    for i, fname in enumerate(feat_names):
        root = find(i)
        clusters.setdefault(root, []).append(fname)

    # 3-F. Choose representative per cluster (keep all control OHE columns)
    keep_idxs = []
    for members in clusters.values():
        if any(m.startswith("cat__") for m in members):
            # keep all control dummy columns in that cluster
            keep_idxs.extend([feat_idx[m] for m in members])
        else:
            best = max(members, key=lambda m: abs_shap_global[feat_idx[m]])
            keep_idxs.append(feat_idx[best])

    keep_idxs = sorted(set(keep_idxs))
    keep_feat_names = [feat_names[i] for i in keep_idxs]

    # map back to original env feature names (strip "num__" prefix)
    kept_env_original = sorted({n.split("__")[1] for n in keep_feat_names if n.startswith("num__")})

    cluster_kept_feats[y] = kept_env_original
    cluster_details[y] = {
        feat_names[i]: cooccur[i].max() / N_BOOTSTRAPS
        for i in range(n_feat)
    }
    constant_stats[y] = {
        feat_names[i]: int(constant_counter[i])
        for i in range(n_feat)
    }

    # 3-G. Final XGBoost using controls + kept env features
    final_preproc = ColumnTransformer(
        [("cat", cat_ohe, control_cols),
         ("num", num_scaler, kept_env_original)],
        remainder="drop"
    )

    X_train_final = final_preproc.fit_transform(train_df)
    X_val_final = final_preproc.transform(val_df)

    xgb_final = XGBRegressor(
        n_estimators=300, learning_rate=0.05,
        max_depth=4, subsample=0.9, colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=114, n_jobs=-1
    )
    xgb_final.fit(X_train_final, train_df[y])

    train_pred = xgb_final.predict(X_train_final)
    val_pred = xgb_final.predict(X_val_final)

    cluster_metrics[y] = {
        "train_R2": r2_score(train_df[y], train_pred),
        "val_R2":   r2_score(val_df[y], val_pred),
        "#kept":    len(kept_env_original)
    }

In [101]:
# === 4. Reporting ===
rows_cluster = []
for y in target_cols:
    rows_cluster.append({
        "Target":          y,
        "M1_val_R2":       M0_metrics[y]["val_R2"],  # assumed original baseline
        "SHAPcluster_R2":  cluster_metrics[y]["val_R2"],
        "# kept":          cluster_metrics[y]["#kept"],
        "Kept features":   ", ".join(cluster_kept_feats[y]) or "-"
    })

print("SHAP-OPTICS feature-cluster selection with XGBoost final model (validation R²)")
pd.DataFrame(rows_cluster)

SHAP-OPTICS feature-cluster selection with XGBoost final model (validation R²)


,Target,M1_val_R2,SHAPcluster_R2,# kept,Kept features
0,N_in,0.854356,0.999994,13,"PAR_std, PAR_x_gwet_root_std, UVA_UVB_ratio_std, UVB_div_gwet_top_std, gwet_gradient_std, gwet_root_std, high_UV_days_pct, rh_max_std, rh_min_mean, temp_min_below_10_pct, thi_95p, vpd_upper_mean, wind_speed_mean"
1,N_out,0.731878,0.957884,33,"C4_PAR_active_days_pct, N_NE_pct, N_NW_pct, PAR_fraction_median, PAR_std, S_SE_pct, UVA_UVB_ratio_mean, UVA_UVB_ratio_std, UVA_mean, UVB_div_gwet_top_mean, UVB_std, UVB_total, UV_idx_mean, W_NW_pct, gdd_total, gwet_prof_5p, gwet_prof_mean, gwet_prof_std, gwet_top_mean, gwet_top_std, heavy_rain_days_pct, high_humidity_pct, max_consecutive_no_rain_days, rain_intensity_index, rh_max_std, temp_max_95p, temp_max_std, temp_min_below_5_pct, temp_range_above_15_pct, temp_range_above_20_pct, thi_5p, vpd_middle_mean, vpd_stressed_pct"
2,P_in,0.651724,0.999916,4,"C3_PAR_active_days_pct, UVB_div_gwet_top_mean, gwet_root_std, rain_intensity_index"
3,P_out,0.824608,0.919543,34,"PAR_fraction_median, PAR_fraction_std, S_SE_pct, UVA_UVB_ratio_std, UVA_std, UVA_total, UVB_std, calm_days_pct, gwet_prof_95p, gwet_prof_mean, gwet_prof_std, gwet_root_95p, gwet_root_mean, gwet_root_std, gwet_top_5p, gwet_top_95p, gwet_top_std, high_UV_days_pct, high_humidity_pct, low_humidity_pct, rainy_days_pct, rh_range_std, temp_max_95p, temp_max_median, temp_max_std, temp_min_5p, temp_min_below_10_pct, temp_min_median, temp_range_mean, temp_range_median, thi_5p, thi_95p, wind_speed_mean, wind_speed_std"
4,K_in,0.817720,0.999956,11,"C3_PAR_active_days_pct, PAR_x_gwet_root_mean, S_SE_pct, gdd_mean_daily, gwet_gradient_mean, max_consecutive_dry_days, max_consecutive_no_rain_days, rain_intensity_index, rh_min_std, thi_95p, vpd_lower_mean"
5,K_out,0.619455,0.934375,31,"N_NE_pct, PAR_fraction_median, PAR_std, UVA_UVB_ratio_mean, UVA_UVB_ratio_median, UVA_UVB_ratio_std, UVB_div_gwet_top_mean, UVB_std, UVB_total, W_NW_pct, calm_days_pct, gwet_prof_5p, gwet_prof_95p, gwet_prof_std, gwet_root_95p, gwet_root_median, gwet_top_5p, gwet_top_median, high_humidity_pct, rainfall_mean, rh_max_std, rh_min_std, temp_max_above_30_pct, temp_max_std, temp_range_above_10_pct, temp_range_mean, temp_range_median, thi_5p, thi_moderate_pct, vpd_stressed_pct, vpd_upper_mean"
6,S_in,0.835257,0.999228,31,"PAR_fraction_std, UVA_UVB_ratio_mean, UVA_UVB_ratio_median, UVA_UVB_ratio_std, UVB_div_gwet_top_mean, UVB_std, UV_idx_std, calm_days_pct, gwet_gradient_mean, gwet_gradient_std, gwet_prof_95p, gwet_prof_mean, gwet_prof_std, gwet_root_mean, gwet_top_95p, gwet_top_median, heavy_rain_days_pct, high_humidity_pct, rainy_days_pct, rh_min_mean, rh_range_std, temp_max_mean, temp_max_median, temp_min_below_10_pct, temp_min_std, temp_range_above_20_pct, temp_range_std, thi_mild_pct, vpd_stressed_pct, wind_speed_mean, wind_speed_std"
7,S_out,0.810077,0.929007,36,"C4_PAR_active_days_pct, N_NE_pct, PAR_fraction_median, UVA_UVB_ratio_median, UVA_UVB_ratio_std, UVA_total, UVB_div_gwet_top_std, UVB_total, UV_idx_mean, UV_idx_median, W_NW_pct, gwet_prof_5p, gwet_prof_95p, gwet_prof_std, gwet_root_5p, gwet_root_std, gwet_top_5p, gwet_top_median, gwet_top_std, rainfall_mean, rainy_days_pct, rh_max_std, rh_min_std, rh_range_std, temp_max_above_30_pct, temp_max_median, temp_max_std, temp_min_median, temp_range_above_15_pct, temp_range_mean, temp_range_median, thi_mean, thi_mild_pct, thi_moderate_pct, vpd_stressed_pct, vpd_upper_mean"


In [102]:
def get_intra_inter_cooccurrence(clusters, cooccur, feat_names, n_bootstraps):
    """
    Return two arrays: intra-cluster normalized co-occurrence vs inter-cluster.
    cooccur is raw counts (shape n_feat x n_feat), feat_names is list/array.
    """
    feat_idx = {f: i for i, f in enumerate(feat_names)}
    intra, inter = [], []
    # build a mapping from feature to its cluster root for fast lookup
    member_to_root = {}
    for root, members in clusters.items():
        for m in members:
            member_to_root[m] = root

    all_feats = list(feat_names)
    for a, b in combinations(all_feats, 2):
        ia, ib = feat_idx[a], feat_idx[b]
        norm_val = cooccur[ia, ib] / n_bootstraps  # in [0,1]
        if member_to_root.get(a, None) is not None and member_to_root.get(b, None) is not None \
           and member_to_root[a] == member_to_root[b]:
            intra.append(norm_val)
        else:
            inter.append(norm_val)
    return np.array(intra), np.array(inter)

def plot_intra_inter(intra, inter, title=None):
    """
    Box/violin comparison of intra vs inter co-occurrence.
    """
    fig, ax = plt.subplots(figsize=(5, 4))
    ax.boxplot([intra, inter], labels=["intra-cluster", "inter-cluster"], patch_artist=True)
    ax.set_ylabel("Normalized co-occurrence")
    ax.set_title(title or "Intra vs Inter co-occurrence")
    ax.grid(False)
    # annotate medians
    medians = [np.median(intra), np.median(inter)]
    for i, m in enumerate(medians, start=1):
        ax.text(i, m + 0.02, f"{m:.2f}", ha="center", va="bottom", fontsize=9)
    plt.tight_layout()
    plt.show()

## 2-3 Compare filter results

In [104]:
methods = {
    "LinearRFE": HIER_RFE_features,
    "LassoRFE": HIER_RFE_LASSO_features,
    "RFRFE": HIER_RFE_RF_features,
    "SHAPCluster": cluster_kept_feats
}
metrics = {
    "LinearRFE": HIER_RFE_metrics,
    "LassoRFE": HIER_RFE_LASSO_metrics,
    "RFRFE": HIER_RFE_RF_metrics,
    "SHAPCluster": cluster_metrics
}

rows = []
for y in target_cols:
    for name in methods:
        feats = methods[name][y]
        m = metrics[name][y]
        rows.append({
            "target": y,
            "method": name,
            "train_R2": m["train_R2"],
            "val_R2": m["val_R2"],
            "n_features": len(feats),
            "features": ", ".join(sorted(feats))
        })
summary_df = pd.DataFrame(rows)
print(summary_df)


   target       method  train_R2    val_R2  n_features  \
0    N_in    LinearRFE  0.897818  0.848606           1   
1    N_in     LassoRFE  0.897341  0.853112           1   
2    N_in        RFRFE  0.999991  0.999977           3   
3    N_in  SHAPCluster  0.999997  0.999994          13   
4   N_out    LinearRFE  0.853902  0.757574          38   
5   N_out     LassoRFE  0.851360  0.758032           5   
6   N_out        RFRFE  0.981839  0.958504           2   
7   N_out  SHAPCluster  0.980879  0.957884          33   
8    P_in    LinearRFE  0.785105  0.653936           1   
9    P_in     LassoRFE  0.784990  0.651445           1   
10   P_in        RFRFE  0.999972  0.999928           5   
11   P_in  SHAPCluster  0.999963  0.999916           4   
12  P_out    LinearRFE  0.857504  0.858049           5   
13  P_out     LassoRFE  0.868984  0.843461          14   
14  P_out        RFRFE  0.958765  0.915158           1   
15  P_out  SHAPCluster  0.959638  0.919543          34   
16   K_in    L

In [105]:
# Calculate pairwise Jaccard overlap
from itertools import combinations

overlap_rows = []
for y in target_cols:
    for a, b in combinations(methods.keys(), 2):
        set_a = set(methods[a][y])
        set_b = set(methods[b][y])
        inter = len(set_a & set_b)
        union = len(set_a | set_b)
        ji = inter / union if union else 1.0
        overlap_rows.append({
            "target": y,
            "method_a": a,
            "method_b": b,
            "jaccard": ji
        })
overlap_df = pd.DataFrame(overlap_rows)
print(overlap_df)

   target   method_a     method_b   jaccard
0    N_in  LinearRFE     LassoRFE  0.000000
1    N_in  LinearRFE        RFRFE  0.333333
2    N_in  LinearRFE  SHAPCluster  0.000000
3    N_in   LassoRFE        RFRFE  0.333333
4    N_in   LassoRFE  SHAPCluster  0.000000
5    N_in      RFRFE  SHAPCluster  0.066667
6   N_out  LinearRFE     LassoRFE  0.102564
7   N_out  LinearRFE        RFRFE  0.052632
8   N_out  LinearRFE  SHAPCluster  0.775000
9   N_out   LassoRFE        RFRFE  0.000000
10  N_out   LassoRFE  SHAPCluster  0.151515
11  N_out      RFRFE  SHAPCluster  0.060606
12   P_in  LinearRFE     LassoRFE  1.000000
13   P_in  LinearRFE        RFRFE  0.200000
14   P_in  LinearRFE  SHAPCluster  0.000000
15   P_in   LassoRFE        RFRFE  0.200000
16   P_in   LassoRFE  SHAPCluster  0.000000
17   P_in      RFRFE  SHAPCluster  0.500000
18  P_out  LinearRFE     LassoRFE  0.187500
19  P_out  LinearRFE        RFRFE  0.000000
20  P_out  LinearRFE  SHAPCluster  0.147059
21  P_out   LassoRFE        RFRF

In [106]:
# Build feature groups per target according to the specified core / full sets logic.

# Assumed existing dictionaries from earlier steps:
# - HIER_RFE_features            : hierarchical RFE using plain LinearRegression
# - HIER_RFE_LASSO_features      : hierarchical RFE using Lasso
# - HIER_RFE_RF_features         : hierarchical RFE using RandomForest (RFRFE)
# - cluster_kept_feats          : SHAP-cluster retained env features (using XGBoost)
# Each is a dict mapping target name to a list of selected env feature names.

# We'll normalize to sets for easy intersection/union.
linear_rfe_sets = {y: set(HIER_RFE_features[y]) for y in target_cols}          # LinearRFE
rf_rfe_sets     = {y: set(HIER_RFE_RF_features[y]) for y in target_cols}       # RFRFE
shap_sets       = {y: set(cluster_kept_feats[y]) for y in target_cols}         # SHAPClusterXGB

# Prepare container for the constructed groups
feature_groups = {}

for y in target_cols:
    groups = {}

    if y == "N_in":
        # core: RFRFE
        groups["core"] = rf_rfe_sets[y]                     # core supported by RF RFE
        # full SHAP cluster set
        groups["full_SHAP"] = shap_sets[y]                  # all SHAP clustering selected features
    
    elif y == "N_out":
        # core: intersection of RFRFE and SHAPCluster
        groups["core"] = rf_rfe_sets[y] & shap_sets[y]
        # full sets
        groups["full_RFRFE"] = rf_rfe_sets[y]               # all RF-RFE-selected features
        groups["full_SHAP"] = shap_sets[y]
    
    elif y == "P_in":
        # core: intersection of RFRFE and SHAPCluster
        groups["core"] = rf_rfe_sets[y] & shap_sets[y]      # core supported by RF RFE and SHAP cluster
        # full sets
        groups["full_RFRFE"] = rf_rfe_sets[y]
        groups["full_SHAP"] = shap_sets[y]
    
    elif y == "P_out":
        # core: intersection of RFRFE
        groups["core"] = rf_rfe_sets[y] & shap_sets[y]
        # full sets
        groups["full_RFRFE"] = rf_rfe_sets[y]
        groups["full_SHAP"] = shap_sets[y]
    
    elif y == "K_in":
        # core: RFRFE 
        groups["core"] = rf_rfe_sets[y] & shap_sets[y]
        # full sets
        groups["full_SHAP"] = shap_sets[y]

    elif y == "K_out":
        # core: intersection of RFRFE and SHAPCluster
        groups["core"] = rf_rfe_sets[y] & shap_sets[y]
        # full sets
        groups["full_RFRFE"] = rf_rfe_sets[y]
        groups["full_SHAP"] = shap_sets[y]
        
    elif y == "S_in":
        # core: intersection of RFRFE and SHAPCluster
        groups["core"] = rf_rfe_sets[y] & shap_sets[y]
        # full sets
        groups["full_RFRFE"] = rf_rfe_sets[y]
        groups["full_SHAP"] = shap_sets[y]

    elif y == "S_out":
        # core: intersection of RFRFE and SHAPCluster
        groups["core"] = rf_rfe_sets[y] & shap_sets[y]
        # full sets
        groups["full_RFRFE"] = rf_rfe_sets[y]
        groups["full_SHAP"] = shap_sets[y]
        
    else:
        # fallback: empty
        groups["core"] = set()
        groups["full_SHAP"] = shap_sets.get(y, set())
        groups["full_RFRFE"] = rf_rfe_sets.get(y, set())

    feature_groups[y] = groups

# Create a summary DataFrame showing sizes and source info
rows = []
for y, grp in feature_groups.items():
    row = {"target": y}
    # core
    row["core_size"] = len(grp.get("core", []))
    row["core_features"] = ", ".join(sorted(grp.get("core", [])))
    # full SHAP
    row["full_SHAP_size"] = len(grp.get("full_SHAP", []))
    row["full_SHAP_features"] = ", ".join(sorted(grp.get("full_SHAP", [])))
    # full RFRFE if exists
    if "full_RFRFE" in grp:
        row["full_RFRFE_size"] = len(grp.get("full_RFRFE", []))
        row["full_RFRFE_features"] = ", ".join(sorted(grp.get("full_RFRFE", [])))
    rows.append(row)

feature_group_summary = pd.DataFrame(rows)
print("Constructed feature groups per target (core / full_SHAP / full_RFRFE where applicable):")
feature_group_summary

Constructed feature groups per target (core / full_SHAP / full_RFRFE where applicable):


,target,core_size,core_features,full_SHAP_size,full_SHAP_features,full_RFRFE_size,full_RFRFE_features
0,N_in,3,"UVA_UVB_ratio_std, gdd_total, thi_5p",13,"PAR_std, PAR_x_gwet_root_std, UVA_UVB_ratio_std, UVB_div_gwet_top_std, gwet_gradient_std, gwet_root_std, high_UV_days_pct, rh_max_std, rh_min_mean, temp_min_below_10_pct, thi_95p, vpd_upper_mean, wind_speed_mean",NaN,NaN
1,N_out,2,"UVA_UVB_ratio_std, vpd_middle_mean",33,"C4_PAR_active_days_pct, N_NE_pct, N_NW_pct, PAR_fraction_median, PAR_std, S_SE_pct, UVA_UVB_ratio_mean, UVA_UVB_ratio_std, UVA_mean, UVB_div_gwet_top_mean, UVB_std, UVB_total, UV_idx_mean, W_NW_pct, gdd_total, gwet_prof_5p, gwet_prof_mean, gwet_prof_std, gwet_top_mean, gwet_top_std, heavy_rain_days_pct, high_humidity_pct, max_consecutive_no_rain_days, rain_intensity_index, rh_max_std, temp_max_95p, temp_max_std, temp_min_below_5_pct, temp_range_above_15_pct, temp_range_above_20_pct, thi_5p, vpd_middle_mean, vpd_stressed_pct",2.0,"UVA_UVB_ratio_std, vpd_middle_mean"
2,P_in,3,"C3_PAR_active_days_pct, UVB_div_gwet_top_mean, gwet_root_std",4,"C3_PAR_active_days_pct, UVB_div_gwet_top_mean, gwet_root_std, rain_intensity_index",5.0,"C3_PAR_active_days_pct, C4_PAR_active_days_pct, UVB_div_gwet_top_mean, gdd_total, gwet_root_std"
3,P_out,1,temp_min_median,34,"PAR_fraction_median, PAR_fraction_std, S_SE_pct, UVA_UVB_ratio_std, UVA_std, UVA_total, UVB_std, calm_days_pct, gwet_prof_95p, gwet_prof_mean, gwet_prof_std, gwet_root_95p, gwet_root_mean, gwet_root_std, gwet_top_5p, gwet_top_95p, gwet_top_std, high_UV_days_pct, high_humidity_pct, low_humidity_pct, rainy_days_pct, rh_range_std, temp_max_95p, temp_max_median, temp_max_std, temp_min_5p, temp_min_below_10_pct, temp_min_median, temp_range_mean, temp_range_median, thi_5p, thi_95p, wind_speed_mean, wind_speed_std",1.0,temp_min_median
4,K_in,2,"C3_PAR_active_days_pct, gdd_mean_daily",11,"C3_PAR_active_days_pct, PAR_x_gwet_root_mean, S_SE_pct, gdd_mean_daily, gwet_gradient_mean, max_consecutive_dry_days, max_consecutive_no_rain_days, rain_intensity_index, rh_min_std, thi_95p, vpd_lower_mean",NaN,NaN
5,K_out,1,UVA_UVB_ratio_std,31,"N_NE_pct, PAR_fraction_median, PAR_std, UVA_UVB_ratio_mean, UVA_UVB_ratio_median, UVA_UVB_ratio_std, UVB_div_gwet_top_mean, UVB_std, UVB_total, W_NW_pct, calm_days_pct, gwet_prof_5p, gwet_prof_95p, gwet_prof_std, gwet_root_95p, gwet_root_median, gwet_top_5p, gwet_top_median, high_humidity_pct, rainfall_mean, rh_max_std, rh_min_std, temp_max_above_30_pct, temp_max_std, temp_range_above_10_pct, temp_range_mean, temp_range_median, thi_5p, thi_moderate_pct, vpd_stressed_pct, vpd_upper_mean",3.0,"UVA_UVB_ratio_std, temp_max_mean, temp_min_mean"
6,S_in,21,"PAR_fraction_std, UVA_UVB_ratio_mean, UVA_UVB_ratio_median, UVA_UVB_ratio_std, UVB_div_gwet_top_mean, UVB_std, calm_days_pct, gwet_prof_95p, gwet_prof_mean, gwet_root_mean, gwet_top_95p, gwet_top_median, heavy_rain_days_pct, rainy_days_pct, rh_min_mean, rh_range_std, temp_max_median, temp_min_std, temp_range_std, vpd_stressed_pct, wind_speed_mean",31,"PAR_fraction_std, UVA_UVB_ratio_mean, UVA_UVB_ratio_median, UVA_UVB_ratio_std, UVB_div_gwet_top_mean, UVB_std, UV_idx_std, calm_days_pct, gwet_gradient_mean, gwet_gradient_std, gwet_prof_95p, gwet_prof_mean, gwet_prof_std, gwet_root_mean, gwet_top_95p, gwet_top_median, heavy_rain_days_pct, high_humidity_pct, rainy_days_pct, rh_min_mean, rh_range_std, temp_max_mean, temp_max_median, temp_min_below_10_pct, temp_min_std, temp_range_above_20_pct, temp_range_std, thi_mild_pct, vpd_stressed_pct, wind_speed_mean, wind_speed_std",32.0,"PAR_fraction_std, UVA_UVB_ratio_mean, UVA_UVB_ratio_median, UVA_UVB_ratio_std, UVB_div_gwet_top_mean, UVB_std, UVB_total, calm_days_pct, gdd_total, gwet_prof_95p, gwet_prof_mean, gwet_root_95p, gwet_root_mean, gwet_top_5p, gwet_top_95p, gwet_top_median, heavy_rain_days_pct, low_humidity_pct, max_consecutive_no_rain_days, rainy_days_pct, rh_min_mean, rh_range_std, temp_max_median, temp_max_std, temp_min_5p, temp_min_std, temp_ra

# 3. Final models

## 3-1 XGBoost

In [109]:
# Helper to build X and y for a given target and feature subset
def build_feature_matrix(df, target, extra_env_feats):
    """
    df: DataFrame containing control_cols, env features, target
    extra_env_feats: set/list of environmental features to include (does not include control)
    returns X_df (with control + those env), y_series
    """
    use_feats = control_cols + list(extra_env_feats)
    X = df[use_feats].copy()
    y = df[target].copy()
    return X, y

# Preprocessor: one-hot encode control categorical columns; environmental features are assumed numeric already
def make_preprocessor_for_env(control_cols, env_feats):
    transformers = []
    # one-hot control columns (use sparse_output=False to get dense)
    transformers.append((
        "cat_ctrl",
        OneHotEncoder(handle_unknown="ignore", sparse_output=False),
        control_cols
    ))
    # pass through env features if any
    if env_feats:
        transformers.append((
            "env",
            "passthrough",
            env_feats
        ))
    return ColumnTransformer(transformers, remainder="drop")

# Parameter distributions for random search
param_dist = {
    "xgb__n_estimators": randint(100, 501),          # 100 to 500
    "xgb__max_depth": randint(3, 8),                 # 3 to 7
    "xgb__learning_rate": loguniform(0.01, 0.3),      # 0.01 to 0.3
    "xgb__subsample": uniform(0.6, 0.4),             # 0.6 to 1.0
    "xgb__colsample_bytree": uniform(0.6, 0.4),      # 0.6 to 1.0
    "xgb__gamma": uniform(0, 1),                     # regularization
    "xgb__reg_alpha": loguniform(1e-3, 10),
    "xgb__reg_lambda": loguniform(1e-3, 10),
}

# Storage for summary
summary_rows = []

# For each target, build three predictor sets and run tuning
for y in target_cols:
    # Prepare feature sets
    core_feats = feature_groups[y]["core"]
    full_shap_feats = feature_groups[y]["full_SHAP"]
    # Baseline: only control
    sets_to_try = {
        "control_only": set(),  # no extra env
        "control_plus_core": core_feats,
        "control_plus_full_SHAP": full_shap_feats
    }

    for set_name, extra_feats in sets_to_try.items():
        # skip if core is empty (e.g., possible) or full_SHAP missing
        if set_name == "control_plus_core" and not extra_feats:
            continue
        if set_name == "control_plus_full_SHAP" and not extra_feats:
            continue

        # Build training and validation matrices
        X_train_raw, y_train = build_feature_matrix(train_df, y, extra_feats)
        X_val_raw, y_val = build_feature_matrix(val_df, y, extra_feats)

        # Preprocess: encode controls and pass through env
        preproc = make_preprocessor_for_env(control_cols, list(extra_feats))
        pipe = Pipeline([
            ("prep", preproc),
            ("xgb", XGBRegressor(
                objective="reg:squarederror",
                tree_method="auto",
                random_state=114,
                n_jobs=-1,
                # will be overridden by RandomizedSearchCV
            ))
        ])

        # Randomized search with 5-fold CV on training set
        search = RandomizedSearchCV(
            estimator=pipe,
            param_distributions=param_dist,
            n_iter=60,  # budget, adjust if too slow
            scoring="r2",
            cv=KFold(n_splits=5, shuffle=True, random_state=114),
            verbose=1,
            random_state=114,
            n_jobs=-1,
            refit=True
        )
        search.fit(X_train_raw, y_train)

        best_pipe = search.best_estimator_

        # Predictions
        train_pred = best_pipe.predict(X_train_raw)
        val_pred = best_pipe.predict(X_val_raw)

        train_r2 = r2_score(y_train, train_pred)
        val_r2 = r2_score(y_val, val_pred)

        # Complexity: number of original X predictors used (control + env)
        n_control = len(control_cols)
        n_env = len(extra_feats)
        total_features = n_control + n_env

        # Record which features (original) are used
        if set_name == "control_only":
            used_features = control_cols.copy()
        else:
            used_features = control_cols + sorted(extra_feats)

        summary_rows.append({
            "target": y,
            "predictor_set": set_name,
            "train_R2": train_r2,
            "val_R2": val_r2,
            "#original_features": total_features,
            "used_features": ", ".join(used_features),
            "best_params": search.best_params_,
        })

# Compile summary DataFrame
xgb_summary = pd.DataFrame(summary_rows)

# Sort for readability
xgb_summary = xgb_summary.sort_values(["target", "predictor_set"])

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 

In [110]:
# Print comparison
print("XGBoost comparison (control, core, full_SHAP) per target:")
print(xgb_summary[[
    "target", "predictor_set", "train_R2", "val_R2", "#original_features"
]].to_string(index=False))

XGBoost comparison (control, core, full_SHAP) per target:
target          predictor_set  train_R2   val_R2  #original_features
  K_in           control_only  0.999571 0.999273                   4
  K_in      control_plus_core  0.999954 0.999912                   6
  K_in control_plus_full_SHAP  0.999971 0.999942                  15
 K_out           control_only  0.865082 0.778705                   4
 K_out      control_plus_core  0.946429 0.933730                   5
 K_out control_plus_full_SHAP  0.951717 0.933489                  35
  N_in           control_only  0.999831 0.999652                   4
  N_in      control_plus_core  0.999994 0.999991                   7
  N_in control_plus_full_SHAP  0.999993 0.999988                  17
 N_out           control_only  0.934366 0.910938                   4
 N_out      control_plus_core  0.977135 0.958172                   6
 N_out control_plus_full_SHAP  0.978156 0.956730                  37
  P_in           control_only  0.996422 0.992

In [111]:
# === Extract & retrain the chosen best XGBoost pipelines (control_plus_core per target) ===
best_xgb_models = {}   # will hold final fitted pipelines per target
best_model_info = []   # summary info

for y in target_cols:
    # filter summary for the chosen predictor_set
    row = xgb_summary[
        (xgb_summary["target"] == y) &
        (xgb_summary["predictor_set"] == "control_plus_core")
    ]
    if row.empty:
        continue  # skip if missing

    row = row.iloc[0]  # single
    extra_feats = feature_groups[y]["core"]  # core set as per logic
    X_train_raw, y_train = build_feature_matrix(train_df, y, extra_feats)
    X_val_raw, y_val = build_feature_matrix(val_df, y, extra_feats)

    # rebuild preprocessor & pipeline with best params
    preproc = make_preprocessor_for_env(control_cols, list(extra_feats))
    # prepare param dict for XGBRegressor: strip "xgb__" prefix if present
    raw_best = row["best_params"]
    xgb_params = {}
    for k, v in raw_best.items():
        if k.startswith("xgb__"):
            xgb_params[k.replace("xgb__", "")] = v
        else:
            xgb_params[k] = v

    # ensure objective and randomness consistent
    xgb_params.setdefault("objective", "reg:squarederror")
    xgb_params.setdefault("random_state", 114)
    xgb_params.setdefault("n_jobs", -1)

    # build final pipeline
    final_pipe = Pipeline([
        ("prep", preproc),
        ("xgb", XGBRegressor(**xgb_params))
    ])
    final_pipe.fit(X_train_raw, y_train)

    # evaluate
    train_pred = final_pipe.predict(X_train_raw)
    val_pred = final_pipe.predict(X_val_raw)
    train_r2 = r2_score(y_train, train_pred)
    val_r2 = r2_score(y_val, val_pred)
    n_features = len(control_cols) + len(extra_feats)

    best_xgb_models[y] = final_pipe
    best_model_info.append({
        "target": y,
        "predictor_set": "control_plus_core",
        "train_R2": train_r2,
        "val_R2": val_r2,
        "#original_features": n_features,
        "core_size": len(extra_feats),
        "used_features": ", ".join(sorted(list(control_cols) + list(extra_feats))),
        "xgb_params": xgb_params
    })

# summary dataframe
best_xgb_summary = pd.DataFrame(best_model_info)
print("Final selected XGBoost models (control_plus_core) performance:")
print(best_xgb_summary[["target", "train_R2", "val_R2", "#original_features", "core_size"]].to_string(index=False))


Final selected XGBoost models (control_plus_core) performance:
target  train_R2   val_R2  #original_features  core_size
  N_in  0.999994 0.999991                   7          3
 N_out  0.977135 0.958172                   6          2
  P_in  0.999951 0.999913                   7          3
 P_out  0.941537 0.927849                   5          1
  K_in  0.999954 0.999912                   6          2
 K_out  0.946429 0.933730                   5          1
  S_in  0.999360 0.999091                  25         21
 S_out  0.963529 0.934067                   8          4


## 3-2 Random Forest

In [113]:
# Helper to build X and y
def build_feature_matrix(df, target, extra_env_feats):
    use_feats = control_cols + list(extra_env_feats)
    X = df[use_feats].copy()
    y = df[target].copy()
    return X, y

# Preprocessor: one-hot encode control columns; pass through env features
def make_preprocessor_for_env(control_cols, env_feats):
    transformers = []
    transformers.append((
        "cat_ctrl",
        OneHotEncoder(handle_unknown="ignore", sparse_output=False),
        control_cols
    ))
    if env_feats:
        transformers.append((
            "env",
            "passthrough",
            env_feats
        ))
    return ColumnTransformer(transformers, remainder="drop")

# Random forest hyperparameter search space
rf_param_dist = {
    "rf__n_estimators": randint(100, 501),         # trees
    "rf__max_depth": randint(3, 13),               # depth
    "rf__min_samples_split": randint(2, 11),       # split
    "rf__min_samples_leaf": randint(1, 6),         # leaf
    "rf__max_features": ["sqrt", "log2", 0.5, 0.75, 1.0],  # feature subsampling
}

# Storage
rf_summary_rows = []

for y in target_cols:
    core_feats = feature_groups[y].get("core", set())
    full_rfrfe_feats = feature_groups[y].get("full_RFRFE", set())
    # Define predictor sets based on availability
    sets_to_try = {
        "control_only": set(),
        "control_plus_core": core_feats,
    }
    if full_rfrfe_feats:
        sets_to_try["control_plus_full_RFRFE"] = full_rfrfe_feats

    for set_name, extra_feats in sets_to_try.items():
        if set_name != "control_only" and not extra_feats:
            continue  # skip empty core/full if missing

        # Build data matrices
        X_train_raw, y_train = build_feature_matrix(train_df, y, extra_feats)
        X_val_raw, y_val = build_feature_matrix(val_df, y, extra_feats)

        preproc = make_preprocessor_for_env(control_cols, list(extra_feats))
        pipe = Pipeline([
            ("prep", preproc),
            ("rf", RandomForestRegressor(random_state=114, n_jobs=-1))
        ])

        search = RandomizedSearchCV(
            estimator=pipe,
            param_distributions=rf_param_dist,
            n_iter=60,
            scoring="r2",
            cv=KFold(n_splits=5, shuffle=True, random_state=114),
            verbose=1,
            random_state=114,
            n_jobs=-1,
            refit=True
        )
        search.fit(X_train_raw, y_train)

        best_pipe = search.best_estimator_
        train_pred = best_pipe.predict(X_train_raw)
        val_pred = best_pipe.predict(X_val_raw)
        train_r2 = r2_score(y_train, train_pred)
        val_r2 = r2_score(y_val, val_pred)

        n_control = len(control_cols)
        n_env = len(extra_feats)
        total_features = n_control + n_env

        if set_name == "control_only":
            used_features = control_cols.copy()
        else:
            used_features = control_cols + sorted(extra_feats)

        rf_summary_rows.append({
            "target": y,
            "predictor_set": set_name,
            "train_R2": train_r2,
            "val_R2": val_r2,
            "#original_features": total_features,
            "used_features": ", ".join(used_features),
            "best_params": search.best_params_,
        })

# Compile summary
rf_summary = pd.DataFrame(rf_summary_rows)
rf_summary = rf_summary.sort_values(["target", "predictor_set"])
print("Random Forest comparison (control, core, full_RFRFE) per target:")
print(rf_summary[["target", "predictor_set", "train_R2", "val_R2", "#original_features"]].to_string(index=False))


Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 

In [114]:
# === Extract & retrain the chosen best Random Forest pipelines (highest val_R2 per target) ===
best_rf_models = {}
best_rf_model_info = []

for y in target_cols:
    # select the row with maximum val_R2 for this target
    cand = rf_summary[rf_summary["target"] == y]
    if cand.empty:
        continue
    row = cand.loc[cand["val_R2"].idxmax()]

    ps = row["predictor_set"]
    # determine extra_feats based on predictor_set
    if ps == "control_only":
        extra_feats = []
    elif ps == "control_plus_core":
        extra_feats = list(feature_groups[y]["core"])
    elif ps == "control_plus_full_RFRFE":
        extra_feats = list(feature_groups[y]["full_RFRFE"])
    else:
        raise ValueError(f"Unknown predictor_set {ps}")

    # build feature matrices
    X_train_raw, y_train = build_feature_matrix(train_df, y, extra_feats)
    X_val_raw,   y_val   = build_feature_matrix(val_df,   y, extra_feats)

    # reconstruct preprocessor + pipeline with the stored best_params
    preproc = make_preprocessor_for_env(control_cols, extra_feats)
    raw_best = row["best_params"]
    rf_params = {}
    for k, v in raw_best.items():
        # strip pipeline prefix if present
        key = k.replace("rf__", "") if k.startswith("rf__") else k
        rf_params[key] = v
    rf_params.setdefault("random_state", 114)
    rf_params.setdefault("n_jobs", -1)

    pipe = Pipeline([
        ("prep", preproc),
        ("rf", RandomForestRegressor(**rf_params))
    ])
    pipe.fit(X_train_raw, y_train)

    # re-evaluate
    train_r2 = r2_score(y_train, pipe.predict(X_train_raw))
    val_r2   = r2_score(y_val,   pipe.predict(X_val_raw))
    n_features = len(control_cols) + len(extra_feats)

    best_rf_models[y] = pipe
    best_rf_model_info.append({
        "target":           y,
        "predictor_set":    ps,
        "train_R2":         train_r2,
        "val_R2":           val_r2,
        "#original_features": n_features,
        "extra_feats_count": len(extra_feats),
        "used_features":    ", ".join(control_cols + extra_feats),
        "rf_params":        rf_params
    })

best_rf_summary = pd.DataFrame(best_rf_model_info)
print("Final selected Random Forest models (best val_R2 per target):")
print(best_rf_summary[[
    "target", "predictor_set", "train_R2", "val_R2", "#original_features", "extra_feats_count"
]].to_string(index=False))

Final selected Random Forest models (best val_R2 per target):
target           predictor_set  train_R2   val_R2  #original_features  extra_feats_count
  N_in       control_plus_core  0.999988 0.999984                   7                  3
 N_out control_plus_full_RFRFE  0.977036 0.961181                   6                  2
  P_in control_plus_full_RFRFE  0.999874 0.999621                   9                  5
 P_out       control_plus_core  0.946857 0.929222                   5                  1
  K_in       control_plus_core  0.999970 0.999905                   6                  2
 K_out       control_plus_core  0.958973 0.933881                   5                  1
  S_in control_plus_full_RFRFE  0.998983 0.997741                  36                 32
 S_out control_plus_full_RFRFE  0.966310 0.928877                   9                  5


## 3-4 Neural Networks

In [116]:
# suppress warnings from convergence / max_iter for MLP
warnings.filterwarnings("ignore")

# Helper to build X and y
def build_feature_matrix(df, target, extra_env_feats):
    use_feats = control_cols + list(extra_env_feats)
    X = df[use_feats].copy()
    y = df[target].copy()
    return X, y

# Preprocessor for MLP: one-hot encode control, standardize env features
def make_preprocessor_mlp(control_cols, env_feats):
    transformers = []
    transformers.append((
        "cat_ctrl",
        OneHotEncoder(handle_unknown="ignore", sparse_output=False),
        control_cols
    ))
    if env_feats:
        transformers.append((
            "num_env",
            StandardScaler(),
            env_feats
        ))
    return ColumnTransformer(transformers, remainder="drop")

# MLP hyperparameter distributions / choices
param_dist = {
    "mlp__hidden_layer_sizes": [(32,), (64,), (32,16), (64,32), (128,64)],
    "mlp__alpha": loguniform(1e-5, 1e-2),           # L2 penalty
    "mlp__learning_rate_init": loguniform(1e-3, 1e-1),
    "mlp__activation": ["relu", "tanh"],
}

# Storage
mlp_summary_rows = []

for y in target_cols:
    core_feats = feature_groups[y].get("core", set())
    full_shap_feats = feature_groups[y].get("full_SHAP", set())
    full_rfrfe_feats = feature_groups[y].get("full_RFRFE", set())

    sets_to_try = {
        "control_only": set(),
        "control_plus_core": core_feats,
        "control_plus_full_SHAP": full_shap_feats,
        "control_plus_full_RFRFE": full_rfrfe_feats
    }

    for set_name, extra_feats in sets_to_try.items():
        if set_name != "control_only" and not extra_feats:
            continue  # skip empty
        # Build data
        X_train_raw, y_train = build_feature_matrix(train_df, y, extra_feats)
        X_val_raw, y_val = build_feature_matrix(val_df, y, extra_feats)

        preproc = make_preprocessor_mlp(control_cols, list(extra_feats))
        pipe = Pipeline([
            ("prep", preproc),
            ("mlp", MLPRegressor(
                random_state=114,
                max_iter=10000,
                early_stopping=True,
                learning_rate="adaptive",
                tol=1e-4,
                verbose=False
            ))
        ])

        search = RandomizedSearchCV(
            estimator=pipe,
            param_distributions=param_dist,
            n_iter=40,
            scoring="r2",
            cv=KFold(n_splits=5, shuffle=True, random_state=114),
            verbose=1,
            random_state=114,
            n_jobs=-1,
            refit=True
        )

        search.fit(X_train_raw, y_train)

        best_pipe = search.best_estimator_
        train_pred = best_pipe.predict(X_train_raw)
        val_pred = best_pipe.predict(X_val_raw)
        train_r2 = r2_score(y_train, train_pred)
        val_r2 = r2_score(y_val, val_pred)

        n_control = len(control_cols)
        n_env = len(extra_feats)
        total_features = n_control + n_env

        if set_name == "control_only":
            used_features = control_cols.copy()
        else:
            used_features = control_cols + sorted(extra_feats)

        mlp_summary_rows.append({
            "target": y,
            "predictor_set": set_name,
            "train_R2": train_r2,
            "val_R2": val_r2,
            "#original_features": total_features,
            "used_features": ", ".join(used_features),
            "best_params": search.best_params_,
        })

# Compile summary
mlp_summary = pd.DataFrame(mlp_summary_rows)
mlp_summary = mlp_summary.sort_values(["target", "predictor_set"])
print("MLP (NN) comparison (control, core, full_SHAP, full_RFRFE) per target:")
print(mlp_summary[["target", "predictor_set", "train_R2", "val_R2", "#original_features"]].to_string(index=False))


Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 

In [117]:
# === Finalize and store the chosen best MLP models per target (first/version before adjustment) ===

# Define which predictor_set to use per target (the original best as specified)
chosen_mlp_sets = {
    "N_in": "control_plus_core",
    "N_out": "control_plus_full_RFRFE",
    "P_in": "control_plus_full_SHAP",
    "P_out": "control_plus_full_SHAP",
    "K_in": "control_plus_core",
    "K_out": "control_plus_full_SHAP",
    "S_in": "control_plus_core",
    "S_out": "control_plus_full_SHAP",
}

best_mlp_models = {}
best_mlp_info = []

for y in target_cols:
    set_name = chosen_mlp_sets[y]
    row = mlp_summary[
        (mlp_summary["target"] == y) &
        (mlp_summary["predictor_set"] == set_name)
    ]
    if row.empty:
        continue
    row = row.iloc[0]
    # Determine feature set
    if set_name == "control_only":
        extra_feats = set()
    elif set_name == "control_plus_core":
        extra_feats = feature_groups[y]["core"]
    elif set_name == "control_plus_full_SHAP":
        extra_feats = feature_groups[y]["full_SHAP"]
    else:
        extra_feats = set()

    # Build data
    X_train_raw, y_train = build_feature_matrix(train_df, y, extra_feats)
    X_val_raw, y_val = build_feature_matrix(val_df, y, extra_feats)

    # Reconstruct preprocessor & pipeline with stored best params
    preproc = make_preprocessor_mlp(control_cols, list(extra_feats))

    # Extract the best hyperparameters for MLP (they are keyed as "mlp__..." in RandomizedSearchCV)
    raw_best = row["best_params"]
    mlp_kwargs = {}
    for k, v in raw_best.items():
        if k.startswith("mlp__"):
            mlp_kwargs[k.replace("mlp__", "")] = v

    # Keep consistency with original training: early stopping, adaptive LR, tol, max_iter
    mlp_kwargs.setdefault("random_state", 114)
    mlp_kwargs.setdefault("early_stopping", True)
    mlp_kwargs.setdefault("learning_rate", "adaptive")
    mlp_kwargs.setdefault("tol", 1e-4)
    mlp_kwargs.setdefault("max_iter", 1000)
    mlp_kwargs.setdefault("validation_fraction", 0.1)  # default used earlier unless overridden

    from sklearn.neural_network import MLPRegressor
    final_pipe = Pipeline([
        ("prep", preproc),
        ("mlp", MLPRegressor(**mlp_kwargs))
    ])

    final_pipe.fit(X_train_raw, y_train)

    train_pred = final_pipe.predict(X_train_raw)
    val_pred = final_pipe.predict(X_val_raw)
    train_r2 = r2_score(y_train, train_pred)
    val_r2 = r2_score(y_val, val_pred)

    n_features = len(control_cols) + (0 if set_name == "control_only" else len(extra_feats))
    used_features = control_cols if set_name == "control_only" else control_cols + sorted(extra_feats)

    best_mlp_models[y] = final_pipe
    best_mlp_info.append({
        "target": y,
        "predictor_set": set_name,
        "train_R2": train_r2,
        "val_R2": val_r2,
        "#original_features": n_features,
        "used_features": ", ".join(used_features),
        "mlp_params": mlp_kwargs
    })

best_mlp_summary = pd.DataFrame(best_mlp_info)
print("Final selected MLP models performance:")
print(best_mlp_summary[["target", "predictor_set", "train_R2", "val_R2", "#original_features"]].to_string(index=False))


Final selected MLP models performance:
target           predictor_set  train_R2   val_R2  #original_features
  N_in       control_plus_core  0.999631 0.999664                   7
 N_out control_plus_full_RFRFE  0.934241 0.898015                   4
  P_in  control_plus_full_SHAP  0.999651 0.999246                   8
 P_out  control_plus_full_SHAP  0.922977 0.915939                  38
  K_in       control_plus_core  0.999021 0.998172                   6
 K_out  control_plus_full_SHAP  0.941460 0.934154                  35
  S_in       control_plus_core  0.999385 0.998439                  25
 S_out  control_plus_full_SHAP  0.955559 0.930069                  40


## 3-5 Compare

In [119]:
# === Combine best-model summaries across model types into one comparison table per target ===

# Normalize and tag model type
def tag_and_select(df, model_name):
    df = df.copy()
    df["model_type"] = model_name
    # Ensure consistent column names: predictor_set, train_R2, val_R2, #original_features
    return df[["target", "model_type", "predictor_set", "train_R2", "val_R2", "#original_features"]]

xgb_tbl = tag_and_select(best_xgb_summary, "XGBoost")
rf_tbl  = tag_and_select(best_rf_summary, "RandomForest")
mlp_tbl = tag_and_select(best_mlp_summary, "MLP")

# Combine
comparison = pd.concat([xgb_tbl, rf_tbl, mlp_tbl], ignore_index=True)

# Order for readability
comparison = comparison.sort_values(["target", "model_type"], ascending=[True, True])

# Optional: compute val-train gap
comparison["gap_R2"] = comparison["train_R2"] - comparison["val_R2"]

# Print nicely
print("Summary of selected best models per target:") 
print(comparison.to_string(index=False))

Summary of selected best models per target:
target   model_type           predictor_set  train_R2   val_R2  #original_features    gap_R2
  K_in          MLP       control_plus_core  0.999021 0.998172                   6  0.000849
  K_in RandomForest       control_plus_core  0.999970 0.999905                   6  0.000065
  K_in      XGBoost       control_plus_core  0.999954 0.999912                   6  0.000042
 K_out          MLP  control_plus_full_SHAP  0.941460 0.934154                  35  0.007306
 K_out RandomForest       control_plus_core  0.958973 0.933881                   5  0.025093
 K_out      XGBoost       control_plus_core  0.946429 0.933730                   5  0.012700
  N_in          MLP       control_plus_core  0.999631 0.999664                   7 -0.000033
  N_in RandomForest       control_plus_core  0.999988 0.999984                   7  0.000004
  N_in      XGBoost       control_plus_core  0.999994 0.999991                   7  0.000003
 N_out          MLP contro

# 4. Model ensemble & deployment

In [121]:
# Simple ensemble wrapper for raw DataFrame inputs
class RawEnsemble:
    def __init__(self, pipelines, weights=None):
        self.pipes = pipelines
        self.weights = weights or [1/len(pipelines)] * len(pipelines)
    def predict(self, X_df):
        # each pipe accepts raw X_df
        preds = np.vstack([pipe.predict(X_df) for pipe in self.pipes]).T
        return preds.dot(self.weights)

# 1. Build deployed_models dict
deployed_models = {}

# In-targets: N_in, P_in, K_in, S_in -> XGBoost only
for y in ["N_in", "P_in", "K_in", "S_in"]:
    deployed_models[y] = best_xgb_models[y]

# Out-targets:
# N_out -> RF only
deployed_models["N_out"] = best_rf_models["N_out"]

# P_out -> simple average RF + XGB
deployed_models["P_out"] = RawEnsemble(
    [best_rf_models["P_out"], best_xgb_models["P_out"]]
)

# K_out, S_out -> XGBoost only
for y in ["K_out", "S_out"]:
    deployed_models[y] = best_xgb_models[y]

# 2. Evaluate performance and assemble summary table
rows = []
for y, model in deployed_models.items():
    # predictions
    pred_train = model.predict(train_df)
    pred_val   = model.predict(val_df)
    # compute R²
    train_r2 = r2_score(train_df[y], pred_train)
    val_r2   = r2_score(val_df[y],   pred_val)
    # feature count = controls + core env features
    core_feats = feature_groups[y]["core"]
    n_feats = len(control_cols) + len(core_feats)
    # predictor set label
    if isinstance(model, RawEnsemble):
        ps = "RF + XGB (avg)"
    elif model == best_xgb_models.get(y):
        ps = "XGBoost"
    else:
        ps = "RandomForest"
    rows.append({
        "target": y,
        "predictor_set": ps,
        "train_R2": train_r2,
        "val_R2": val_r2,
        "#original_features": n_feats
    })

summary = pd.DataFrame(rows)
print("Deployed model performance summary:")
print(summary.to_string(index=False))


Deployed model performance summary:
target  predictor_set  train_R2   val_R2  #original_features
  N_in        XGBoost  0.999994 0.999991                   7
  P_in        XGBoost  0.999951 0.999913                   7
  K_in        XGBoost  0.999954 0.999912                   6
  S_in        XGBoost  0.999360 0.999091                  25
 N_out   RandomForest  0.977036 0.961181                   6
 P_out RF + XGB (avg)  0.945309 0.929592                   5
 K_out        XGBoost  0.946429 0.933730                   5
 S_out        XGBoost  0.963529 0.934067                   8


In [122]:
balance_targets = ["N_bal", "P_bal", "K_bal", "S_bal"]
in_targets      = ["N_in",    "P_in",    "K_in",    "S_in"]
out_targets     = ["N_out",   "P_out",   "K_out",   "S_out"]

results = []
for bal, yin, yout in zip(balance_targets, in_targets, out_targets):
    # predicted in/out
    pred_in_train  = deployed_models[yin].predict(train_df)
    pred_out_train = deployed_models[yout].predict(train_df)
    pred_bal_train = pred_in_train - pred_out_train

    pred_in_val  = deployed_models[yin].predict(val_df)
    pred_out_val = deployed_models[yout].predict(val_df)
    pred_bal_val = pred_in_val - pred_out_val

    # true in/out
    true_in_train  = train_df[yin].values
    true_out_train = train_df[yout].values
    true_bal_train = true_in_train - true_out_train

    true_in_val  = val_df[yin].values
    true_out_val = val_df[yout].values
    true_bal_val = true_in_val - true_out_val

    # compute R² on the balance
    train_r2 = r2_score(true_bal_train, pred_bal_train)
    val_r2   = r2_score(true_bal_val,   pred_bal_val)

    results.append({
        "balance": bal,
        "train_R2": train_r2,
        "val_R2":   val_r2
    })

bal_summary = pd.DataFrame(results)
print("Final balance model performance:")
print(bal_summary.to_string(index=False))

Final balance model performance:
balance  train_R2   val_R2
  N_bal  0.932253 0.825084
  P_bal  0.992585 0.981758
  K_bal  0.891301 0.835497
  S_bal  0.932075 0.879855


# Export

In [247]:
df_full.to_csv('df_full.csv', index=False)

In [255]:
train_df.to_csv('train_df.csv', index=False)
val_df.to_csv('val_df.csv', index=False)
test_df.to_csv('test_df.csv', index=False)

In [249]:
# diff_export.py
class DiffModel:
    """Simple unweighted subtraction: in_pred - out_pred"""
    def __init__(self, in_model, out_model):
        self.in_model = in_model
        self.out_model = out_model
    def predict(self, X):
        return self.in_model.predict(X) - self.out_model.predict(X)

for bal in ["N_bal", "P_bal"]:
    yin  = bal.replace("_bal", "_in")
    yout = bal.replace("_bal", "_out")

    dm = DiffModel(
        deployed_models[yin],
        deployed_models[yout]
    )
    fname = f"{bal}_diff_model.pkl"
    joblib.dump(dm, fname)
    print(f"Exported unweighted diff model for {bal} → {fname}")


Exported unweighted diff model for N_bal → N_bal_diff_model.pkl
Exported unweighted diff model for P_bal → P_bal_diff_model.pkl


In [251]:
# export_state.py
joblib.dump(feature_groups,    "feature_groups.pkl")
print("Exported feature_groups.pkl")

Exported feature_groups.pkl
